In [1]:
import io
import json
import os
import re
from datetime import datetime
from typing import Dict, List, Optional, Tuple

import pandas as pd
from google.oauth2.service_account import Credentials
from googleapiclient.discovery import build
from googleapiclient.http import MediaIoBaseDownload


# ======================================================
# 1) Credenciales (Colab usa MI_JSON desde userdata; fuera de Colab usa MI_JSON env)
# ======================================================
def get_credentials() -> Credentials:
    """
    - En Colab: usa MI_JSON desde google.colab.userdata (NO os.environ)
    - En GitHub/local: usa variable de entorno MI_JSON
    """
    info: Dict

    try:
        from google.colab import userdata  # type: ignore

        mi_json = userdata.get("MI_JSON")
        if not mi_json:
            raise ValueError("MI_JSON no encontrado en Colab userdata")
        info = json.loads(mi_json)
        print("Entorno detectado: Google Colab")
    except Exception:
        mi_json = os.environ.get("MI_JSON")
        if not mi_json:
            raise ValueError("MI_JSON no encontrado como variable de entorno (GitHub/local)")
        info = json.loads(mi_json)
        print("Entorno detectado: GitHub / local")

    return Credentials.from_service_account_info(
        info,
        scopes=[
            "https://www.googleapis.com/auth/drive.readonly",
            "https://www.googleapis.com/auth/spreadsheets.readonly",
        ],
    )


creds = get_credentials()
drive_service = build("drive", "v3", credentials=creds)


# ======================================================
# 2) Helpers: meses en español + parsing del nombre del archivo
# ======================================================
MES_MAP = {
    "ene": 1,
    "feb": 2,
    "mar": 3,
    "abr": 4,
    "may": 5,
    "jun": 6,
    "jul": 7,
    "ago": 8,
    "sep": 9,
    "oct": 10,
    "nov": 11,
    "dic": 12,
}
MES_NOMBRE = {
    1: "Enero",
    2: "Febrero",
    3: "Marzo",
    4: "Abril",
    5: "Mayo",
    6: "Junio",
    7: "Julio",
    8: "Agosto",
    9: "Septiembre",
    10: "Octubre",
    11: "Noviembre",
    12: "Diciembre",
}


def sheet_name_from_date(dt: datetime) -> str:
    """Ej: datetime(2025,12,...) -> 'Diciembre 2025' """
    return f"{MES_NOMBRE[dt.month]} {dt.year}"


def parse_range_from_filename(name: str) -> Optional[Tuple[int, int, int, int]]:
    """
    Espera nombres tipo:
      'Asignaciones de Cartera Ene26-Abr26.xlsx'
      'Asignaciones de Cartera Sep25-Dic25.xlsx'
    Retorna (start_year, start_month, end_year, end_month) o None si no matchea.
    """
    m = re.search(r"([A-Za-z]{3})(\d{2})\s*-\s*([A-Za-z]{3})(\d{2})", name, flags=re.IGNORECASE)
    if not m:
        return None

    m1, y1, m2, y2 = m.group(1).lower(), m.group(2), m.group(3).lower(), m.group(4)
    if m1 not in MES_MAP or m2 not in MES_MAP:
        return None

    start_month = MES_MAP[m1]
    end_month = MES_MAP[m2]
    start_year = 2000 + int(y1)
    end_year = 2000 + int(y2)

    return (start_year, start_month, end_year, end_month)


def month_index(year: int, month: int) -> int:
    """Convierte (year, month) a índice comparable."""
    return year * 12 + month


def file_covers_month(file_range: Tuple[int, int, int, int], target_dt: datetime) -> bool:
    sy, sm, ey, em = file_range
    t = month_index(target_dt.year, target_dt.month)
    a = month_index(sy, sm)
    b = month_index(ey, em)
    return a <= t <= b


def is_df_empty_like(df: Optional[pd.DataFrame]) -> bool:
    """
    Considera 'vacía' si:
    - df es None
    - df tiene 0 filas
    - o todas las filas están completamente NaN
    """
    if df is None or df.shape[0] == 0:
        return True
    return df.dropna(how="all").shape[0] == 0


# ======================================================
# 3) Listar archivos en carpeta y elegir el correcto por mes
# ======================================================
def list_assignment_files_in_folder(folder_id: str) -> List[Dict]:
    """
    Lista archivos en la carpeta cuyo nombre contenga 'Asignaciones de Cartera'
    y devuelve una lista con metadatos: id, name, mimeType, modifiedTime, parsed_range
    """
    q = f"'{folder_id}' in parents and trashed=false and name contains 'Asignaciones de Cartera'"
    files: List[Dict] = []
    page_token = None

    while True:
        resp = (
            drive_service.files()
            .list(q=q, fields="nextPageToken, files(id,name,mimeType,modifiedTime)", pageToken=page_token)
            .execute()
        )

        for f in resp.get("files", []):
            fr = parse_range_from_filename(f.get("name", ""))
            if fr:
                f["parsed_range"] = fr
                files.append(f)

        page_token = resp.get("nextPageToken")
        if not page_token:
            break

    if not files:
        raise ValueError(
            "No encontré archivos 'Asignaciones de Cartera' con rango tipo Ene26-Abr26 dentro de la carpeta."
        )

    return files


def pick_file_for_month(files_meta: List[Dict], target_dt: datetime) -> Dict:
    """
    Escoge el archivo cuya ventana (en el nombre) cubra el mes target_dt.
    Si hay varios, elige el de rango más corto (más específico) y si empatan, el más reciente.
    """
    candidates = []
    for f in files_meta:
        fr = f["parsed_range"]
        if file_covers_month(fr, target_dt):
            sy, sm, ey, em = fr
            span = month_index(ey, em) - month_index(sy, sm)
            candidates.append((span, f.get("modifiedTime", ""), f))

    if candidates:
        candidates.sort(key=lambda x: (x[0], x[1]))  # menor span, luego por modifiedTime asc
        min_span = candidates[0][0]
        same_span = [c for c in candidates if c[0] == min_span]
        same_span.sort(key=lambda x: x[1], reverse=True)  # más reciente primero
        return same_span[0][2]

    t = month_index(target_dt.year, target_dt.month)

    past = []
    for f in files_meta:
        sy, sm, ey, em = f["parsed_range"]
        end_i = month_index(ey, em)
        if end_i <= t:
            past.append((end_i, f.get("modifiedTime", ""), f))
    if past:
        past.sort(key=lambda x: (x[0], x[1]), reverse=True)
        return past[0][2]

    future = []
    for f in files_meta:
        sy, sm, ey, em = f["parsed_range"]
        start_i = month_index(sy, sm)
        if start_i >= t:
            future.append((start_i, f.get("modifiedTime", ""), f))
    if future:
        future.sort(key=lambda x: (x[0], x[1]))
        return future[0][2]

    raise ValueError("No se pudo escoger un archivo por fecha (revisa nombres/rangos).")


# ======================================================
# 4) Descargar archivo (Google Sheets o Excel) a memoria
# ======================================================
def download_file_to_buffer(file_id: str, mime_type: str) -> io.BytesIO:
    buffer = io.BytesIO()

    if mime_type == "application/vnd.google-apps.spreadsheet":
        request = drive_service.files().export_media(
            fileId=file_id,
            mimeType="application/vnd.openxmlformats-officedocument.spreadsheetml.sheet",
        )
    else:
        request = drive_service.files().get_media(fileId=file_id)

    downloader = MediaIoBaseDownload(buffer, request)
    done = False
    while not done:
        _, done = downloader.next_chunk()

    buffer.seek(0)
    return buffer


# ======================================================
# 5) Intentar leer hoja del mes target; si está vacía -> retroceder mes a mes
# ======================================================
def shift_month(dt: datetime, n: int) -> datetime:
    """Mueve dt n meses (n puede ser negativo)."""
    y = dt.year + (dt.month - 1 + n) // 12
    m = (dt.month - 1 + n) % 12 + 1
    return datetime(y, m, 1)


def load_assignment_base_from_folder(
    folder_id: str, max_back_months: int = 24
) -> Tuple[pd.DataFrame, Dict, str]:
    """
    Busca base del mes actual; si no existe o está vacía,
    busca mes anterior (incluyendo cambio de archivo si aplica).
    Retorna: (df, file_meta, sheet_name_usada)
    """
    files_meta = list_assignment_files_in_folder(folder_id)
    today = datetime.today()

    last_error: Optional[Exception] = None

    for back in range(0, max_back_months + 1):
        target_dt = shift_month(today, -back)
        target_sheet = sheet_name_from_date(target_dt)
        chosen = pick_file_for_month(files_meta, target_dt)

        try:
            buffer = download_file_to_buffer(chosen["id"], chosen["mimeType"])
            df = pd.read_excel(buffer, sheet_name=target_sheet, engine="openpyxl")

            if is_df_empty_like(df):
                print(f"🟡 {target_sheet} encontrado pero vacío en: {chosen['name']} -> probando mes anterior...")
                continue

            print("✅ Base encontrada")
            print(f"   Archivo: {chosen['name']}")
            print(f"   Hoja:    {target_sheet}")
            return df, chosen, target_sheet

        except Exception as e:
            last_error = e
            print(
                f"🟠 No se pudo usar {target_sheet} en {chosen['name']} ({type(e).__name__}) -> probando mes anterior..."
            )

    raise RuntimeError(
        f"No encontré una hoja válida en los últimos {max_back_months} meses. "
        f"Último error: {repr(last_error)}"
    )


# ======================================================
# 6) EJECUCIÓN
# ======================================================
FOLDER_ID = "1cf2p3R7iM0xowAt4muEruDwxZoZqD_jB"

df, meta_file, sheet_used = load_assignment_base_from_folder(
    folder_id=FOLDER_ID,
    max_back_months=24,
)

df.head()

Entorno detectado: GitHub / local


✅ Base encontrada
   Archivo: Asignaciones de Cartera Ene26-Abr26.xlsx
   Hoja:    Enero 2026


,Referencia,Cedula,Nombre del cliente,Id deuda,correo,Deudas Activas,BANCO,Número de Crédito,Deuda Resuelve,DBT,...,I_ESPERADO,Negociador,Tipo Elegible,P_Cierre_norm,Días Atraso liquidacion,Estado PL,Ultima Liquidacion,dias_desde_ultima_liq,Priority_level,Bucket
0,3012650,52216213,CLAUDIA PATRICIA AVENDANO CALDERON,111266,claudiapa426@hotmail.com,2,Scotiabank Citibank,318121778,9256000.0,58255000.0,...,125415.711006,Negodito,NaN,0.633814,NaN,Al día,NaT,NaN,Prioridad 2,NaN
1,3122962933,50868603,Luz Nelly Betancourt Ortega,225561,zulyllen@hotmail.com,3,Banco de Occidente,540625-6-730934-608 VQ,2138165.0,11396681.0,...,12585.582824,William Santiago Abril Esguerra,NaN,0.392411,2162.0,Atrasado,NaT,NaN,Prioridad 1,10.0
2,3122962933,50868603,Luz Nelly Betancourt Ortega,225562,zulyllen@hotmail.com,3,Banco de Occidente,417899-4-107487-516 VQ,3299116.0,11396681.0,...,18849.499266,William Santiago Abril Esguerra,NaN,0.340089,1796.0,Atrasado,NaT,NaN,Prioridad 1,10.0
3,3122962933,50868603,Luz Nelly Betancourt Ortega,225563,zulyllen@hotmail.com,3,Éxito,745089099,5959400.0,11396681.0,...,32976.814790,William Santiago Abril Esguerra,NaN,0.274687,1342.0,Atrasado,NaT,NaN,Prioridad 1,10.0
4,3196892494,52902886,Diana Marcela Suarez,404243,marcelasuarezladino@outlook.com,5,Banco Popular,346383,1921650.0,10126350.0,...,9581.346900,Dayana Isabel Ojito Ortiz,NaN,0.346250,1173.0,Atrasado,NaT,NaN,Prioridad 1,NaN


In [2]:
df = df.rename(columns={"Deuda Resuelve": "D_BRAVO"})

In [3]:
df.columns

Index(['Referencia', 'Cedula', 'Nombre del cliente', 'Id deuda', 'correo',
       'Deudas Activas', 'BANCO', 'Número de Crédito', 'D_BRAVO', 'DBT',
       'MORA', 'Dias de Atraso', 'Apartado Mensual', 'Fecha inicio',
       'vehiculo', 'Meses en el Programa', 'tipo_cliente', 'Comisión Mensual',
       'Tipo de cobro', 'CE', 'Estado Deuda', 'estado_novacion', 'Fecha PL',
       'sub_estado_deuda', 'ID_reparadora', 'estado_reparadora',
       'sub_estado_reparadora', 'dias_mora_ingreso', 'ultima_actividad',
       'PB_PL', 'estado_estructuracion', 'estado_flujo_liquidacion',
       'Ahorro total', 'Ahorro medio', 'Por cobrar', 'BANCOS_ESTANDAR',
       'Descuento', 'ultimo contacto', 'fecha mensaje', 'Mora_estructurado',
       'MORA_CREDITO', 'Potencial Credito', 'ultimo Pab', 'ultima act',
       'Tipo de Liquidacion', 'Clasificación Banco', 'Apartados 30%', 'Plazo',
       'Estructurable', 'Potencial', '# Pagos', 'P_Pab', 'S_PAb',
       'Potencial_Estructurados', 'Tipo_Liq_calc', 'P_

In [4]:
# ---------------------------------------
# PASO 1 — Construir base de cartera asignada
# ---------------------------------------

# Trabajamos sobre copia por seguridad
df_cartera = df.copy()

# Columnas que queremos conservar
cols_cartera = [
    "Referencia",
    "Id deuda",
    "Cedula",
    "Nombre del cliente",
    "Negociador",
    "BANCOS_ESTANDAR",
    "Descuento",
    "D_BRAVO",
    "MORA",
    "Estructurable",
    "Potencial",
    "Meses en el Programa",
    "Tipo de Liquidacion",
    "Bucket",
    "Ahorro total",
    "Ahorro medio",
    "Por cobrar",
    "Potencial Credito",
    "Estado Deuda",
    "sub_estado_deuda",
    "estado_reparadora",
    "sub_estado_reparadora",
    "Mora_estructurado",
    "MORA_CREDITO",
    "Priority_level",
    "ultimo contacto",
    "fecha mensaje"



]

# Validación: columnas faltantes
faltantes = [c for c in cols_cartera if c not in df_cartera.columns]
if faltantes:
    raise ValueError(f"Faltan estas columnas en df: {faltantes}")

# Nos quedamos solo con esas columnas
df_cartera = df_cartera[cols_cartera].copy()

# Opcional: eliminar duplicados por deuda (clave natural)
df_cartera = df_cartera.drop_duplicates(subset=["Id deuda"])

# Reset index limpio
df_cartera = df_cartera.reset_index(drop=True)

df_cartera.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41258 entries, 0 to 41257
Data columns (total 27 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   Referencia             41258 non-null  int64         
 1   Id deuda               41258 non-null  int64         
 2   Cedula                 41258 non-null  int64         
 3   Nombre del cliente     41258 non-null  object        
 4   Negociador             41258 non-null  object        
 5   BANCOS_ESTANDAR        41258 non-null  object        
 6   Descuento              36308 non-null  float64       
 7   D_BRAVO                41258 non-null  float64       
 8   MORA                   41258 non-null  int64         
 9   Estructurable          41258 non-null  int64         
 10  Potencial              36170 non-null  float64       
 11  Meses en el Programa   41258 non-null  int64         
 12  Tipo de Liquidacion    41258 non-null  object        
 13  B

In [5]:
df_cartera

,Referencia,Id deuda,Cedula,Nombre del cliente,Negociador,BANCOS_ESTANDAR,Descuento,D_BRAVO,MORA,Estructurable,...,Potencial Credito,Estado Deuda,sub_estado_deuda,estado_reparadora,sub_estado_reparadora,Mora_estructurado,MORA_CREDITO,Priority_level,ultimo contacto,fecha mensaje
0,3012650,111266,52216213,CLAUDIA PATRICIA AVENDANO CALDERON,Negodito,Scotiabank Colpatria,0.7126,9256000.0,120,1,...,NaN,negotiation,negotiation,active,saving_paused,No tiene estructurado,No,Prioridad 2,6,NaT
1,3122962933,225561,50868603,Luz Nelly Betancourt Ortega,William Santiago Abril Esguerra,Banco de Occidente,0.5000,2138165.0,55,1,...,NaN,negotiation,negotiation,active,saving_paused,No tiene estructurado,No,Prioridad 1,1,2025-12-29 07:31:41
2,3122962933,225562,50868603,Luz Nelly Betancourt Ortega,William Santiago Abril Esguerra,Banco de Occidente,0.5600,3299116.0,55,1,...,NaN,negotiation,negotiation,active,saving_paused,No tiene estructurado,No,Prioridad 1,1,2025-12-29 07:31:41
3,3122962933,225563,50868603,Luz Nelly Betancourt Ortega,William Santiago Abril Esguerra,Tuya,0.6715,5959400.0,55,0,...,NaN,negotiation,negotiation,active,saving_paused,No tiene estructurado,No,Prioridad 1,1,2025-12-29 07:31:41
4,3196892494,404243,52902886,Diana Marcela Suarez,Dayana Isabel Ojito Ortiz,Banco Popular,0.4800,1921650.0,79,1,...,NaN,negotiation,negotiation,active,saving_paused,No tiene estructurado,No,Prioridad 1,6,NaT
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41253,3225833196,2091518,1019066241,JUAN DAVID LOZANO TRIVINO,Gabriela Saavedra Latorre,Banco Davivienda,0.3637,13695024.0,7,1,...,NaN,new,debt_assignment,active,saving_opening,No tiene estructurado,No,Prioridad 2,0,2026-01-09 09:44:37
41254,3225833196,2091519,1019066241,JUAN DAVID LOZANO TRIVINO,Gabriela Saavedra Latorre,Scotiabank Colpatria,0.5000,50463309.0,7,0,...,NaN,new,debt_assignment,active,saving_opening,No tiene estructurado,No,Prioridad 2,0,2026-01-09 09:44:37
41255,3225833196,2091520,1019066241,JUAN DAVID LOZANO TRIVINO,Gabriela Saavedra Latorre,Bancolombia,0.4700,82022139.0,7,0,...,NaN,new,debt_assignment,active,saving_opening,No tiene estructurado,No,Prioridad 2,0,2026-01-09 09:44:37
41256,3105507630,2091523,79381039,JUAN CARLOS PACHECO CONTRERAS,Julian Andres Ossa Bohorquez,Bancolombia,0.4700,130935011.0,7,0,...,NaN,new,debt_assignment,active,saving_opening,No tiene estructurado,No,Prioridad 2,0,2026-01-09 10:51:57


In [6]:
# -*- coding: utf-8 -*-
import os
import json
import pandas as pd
import gspread
from datetime import datetime
from google.oauth2.service_account import Credentials

# =====================================
# 1. Cargar MI_JSON (Colab / GitHub)
# =====================================
def get_mi_json():
    try:
        # ---- Colab ----
        from google.colab import userdata
        mi_json = userdata.get("MI_JSON")
        if not mi_json:
            raise RuntimeError("MI_JSON no encontrado en Colab userdata")
        print("Entorno detectado: Google Colab")
        return mi_json
    except Exception:
        # ---- GitHub Actions / local ----
        mi_json = os.environ.get("MI_JSON")
        if not mi_json:
            raise RuntimeError("MI_JSON no encontrado como variable de entorno (GitHub/local)")
        if os.environ.get("GITHUB_ACTIONS") == "true":
            print("Entorno detectado: GitHub Actions")
        else:
            print("Entorno detectado: Local")
        return mi_json

mi_json = get_mi_json()
creds_dict = json.loads(mi_json)

SCOPES = ["https://www.googleapis.com/auth/spreadsheets.readonly"]
creds = Credentials.from_service_account_info(creds_dict, scopes=SCOPES)
gc = gspread.authorize(creds)

# =====================================
# 2. Definir año objetivo (regla 6 días)
# =====================================
today = datetime.today()

if today.month == 1 and today.day <= 6:
    target_year = today.year - 1
else:
    target_year = today.year

print(f"📅 Año objetivo detectado: {target_year}")

# =====================================
# 3. Abrir spreadsheet
# =====================================
SPREADSHEET_ID = "1O8OHuVhgwhLw8XYEBf1uBzLYrxQ45rPiZecHOnAa1Go"
sh = gc.open_by_key(SPREADSHEET_ID)

# =====================================
# 4. Buscar hoja por año en el nombre
# =====================================
worksheet_found = None
for ws in sh.worksheets():
    if str(target_year) in ws.title:
        worksheet_found = ws
        break

if worksheet_found is None:
    raise ValueError(
        f"No se encontró ninguna hoja cuyo nombre contenga el año {target_year}"
    )

print(f"✅ Hoja seleccionada: '{worksheet_found.title}'")

# =====================================
# 5. Leer a DataFrame
# =====================================
records = worksheet_found.get_all_records()

df_act = pd.DataFrame(records)

print("✅ df_act cargado correctamente")
print("Shape:", df_act.shape)
print(df_act.head())

Entorno detectado: GitHub Actions
📅 Año objetivo detectado: 2026


✅ Hoja seleccionada: 'act 2026'


✅ df_act cargado correctamente
Shape: (6474, 7)
                                        observations  \
0  8:09 a. m. 2/01/2026 VivianR// Se valida con c...   
1  8:09 a. m. 2/01/2026 VivianR// Se valida con c...   
2  8:09 a. m. 2/01/2026 VivianR// Se valida con c...   
3  8:09 a. m. 2/01/2026 VivianR// Se valida con c...   
4  8:09 a. m. 2/01/2026 VivianR// Se valida con c...   

                               end  debt_id  payment_to_bank  \
0  vivian.rodriguez@gobravo.com.co  1898742  (314820000,COP)   
1  vivian.rodriguez@gobravo.com.co  1898737  (270710000,COP)   
2  vivian.rodriguez@gobravo.com.co  1898741  (313720000,COP)   
3  vivian.rodriguez@gobravo.com.co  1898738  (299530000,COP)   
4  vivian.rodriguez@gobravo.com.co  1898739  (304260000,COP)   

           inserted_at  bank_reference       Status_Act  
0  2026-01-02 13:10:20      3157363121  CONTRAPROPUESTA  
1  2026-01-02 13:10:53      3157363121  CONTRAPROPUESTA  
2  2026-01-02 13:09:48      3157363121  CONTRAPROPUESTA 

In [7]:
df_act

,observations,end,debt_id,payment_to_bank,inserted_at,bank_reference,Status_Act
0,8:09 a. m. 2/01/2026 VivianR// Se valida con c...,vivian.rodriguez@gobravo.com.co,1898742,"(314820000,COP)",2026-01-02 13:10:20,3157363121,CONTRAPROPUESTA
1,8:09 a. m. 2/01/2026 VivianR// Se valida con c...,vivian.rodriguez@gobravo.com.co,1898737,"(270710000,COP)",2026-01-02 13:10:53,3157363121,CONTRAPROPUESTA
2,8:09 a. m. 2/01/2026 VivianR// Se valida con c...,vivian.rodriguez@gobravo.com.co,1898741,"(313720000,COP)",2026-01-02 13:09:48,3157363121,CONTRAPROPUESTA
3,8:09 a. m. 2/01/2026 VivianR// Se valida con c...,vivian.rodriguez@gobravo.com.co,1898738,"(299530000,COP)",2026-01-02 13:11:34,3157363121,CONTRAPROPUESTA
4,8:09 a. m. 2/01/2026 VivianR// Se valida con c...,vivian.rodriguez@gobravo.com.co,1898739,"(304260000,COP)",2026-01-02 13:12:05,3157363121,CONTRAPROPUESTA
...,...,...,...,...,...,...,...
6469,12:00 p. m. || Dayana O. || Se escala a contac...,dayana.ojito@gobravo.com.co,1615331,"(730996600,COP)",2026-01-15 17:02:51,3216367967,CONTRAPROPUESTA
6470,Vanessa Castillo// 15-01-2026 /12: 04 am se ...,wendy.castillo@gobravo.com.co,1797839,"(1000000000,COP)",2026-01-15 17:04:07,3105529539,EN_CREDITO
6471,12:00pm//Daniela Sanchez// Me como a la casa d...,maria.sarta@gobravo.com.co,1819744,"(900000000,COP)",2026-01-15 17:06:01,3123886715,EN_CREDITO
6472,12:06pm//Daniela Sanchez// se recibe dcto por ...,maria.sarta@gobravo.com.co,1819741,"(16200000000,COP)",2026-01-15 17:07:53,3123886715,REVISAR


In [8]:
import pandas as pd
import numpy as np
import re

df_act = df_act.copy()

s = df_act["inserted_at"]

# Pasar todo a string limpio (sin romper NaN)
s_str = s.astype("string").str.strip()

# Serie destino
dt = pd.Series(pd.NaT, index=df_act.index, dtype="datetime64[ns]")

# -----------------------------
# 1) Formato ISO: 2025-12-18 02:09:59 (o 2025-12-18T02:09:59Z)
# -----------------------------
mask_iso = s_str.str.match(r"^\d{4}-\d{2}-\d{2}", na=False)
iso_clean = (
    s_str[mask_iso]
    .str.replace("T", " ", regex=False)
    .str.replace("Z", "", regex=False)
)
dt.loc[mask_iso] = pd.to_datetime(iso_clean, errors="coerce")  # conserva hora

# -----------------------------
# 2) Formato con slash: 2/1/2025 (puede ser d/m/y o m/d/y)
# -----------------------------
mask_slash = s_str.str.contains(r"/", na=False) & (~mask_iso)

# extraer día/mes/año como números
parts = s_str[mask_slash].str.extract(r"^\s*(\d{1,2})/(\d{1,2})/(\d{4})\s*$")
a = pd.to_numeric(parts[0], errors="coerce")  # primera parte
b = pd.to_numeric(parts[1], errors="coerce")  # segunda parte

# reglas para decidir dayfirst vs monthfirst
mask_dayfirst = (a > 12) & (b <= 12)
mask_monthfirst = (b > 12) & (a <= 12)
mask_ambigua = ~(mask_dayfirst | mask_monthfirst)

idx_slash = parts.index

# dayfirst seguro
idx_day = idx_slash[mask_dayfirst.fillna(False)]
dt.loc[idx_day] = pd.to_datetime(s_str.loc[idx_day], errors="coerce", dayfirst=True)

# monthfirst seguro
idx_mon = idx_slash[mask_monthfirst.fillna(False)]
dt.loc[idx_mon] = pd.to_datetime(s_str.loc[idx_mon], errors="coerce", dayfirst=False)

# ambigua -> por defecto dayfirst=True (tu estándar)
idx_amb = idx_slash[mask_ambigua.fillna(True)]
tmp = pd.to_datetime(s_str.loc[idx_amb], errors="coerce", dayfirst=True)

# si alguna ambigua falla, reintenta monthfirst
mask_fail = tmp.isna()
if mask_fail.any():
    tmp.loc[mask_fail] = pd.to_datetime(s_str.loc[idx_amb[mask_fail]], errors="coerce", dayfirst=False)

dt.loc[idx_amb] = tmp

# -----------------------------
# 3) Guardar resultado final
# -----------------------------
df_act["inserted_at"] = dt

# (Opcional) Ver cuántos quedaron NaT
print("NaT en inserted_at:", df_act["inserted_at"].isna().sum())
print("dtype:", df_act["inserted_at"].dtype)

NaT en inserted_at: 0
dtype: datetime64[ns]


In [9]:
import pandas as pd
import numpy as np
import re

df_act["payment_to_bank"] = (
    df_act["payment_to_bank"]
    .astype(str)
    .str.extract(r"\(?\s*([\d]+)\s*,?\s*COP?\s*\)?", expand=False)
    .astype(float)
)

In [10]:
df_act['payment_to_bank'] = df_act['payment_to_bank']/100

In [11]:
df_act = df_act.rename(columns={"Status_Act": "CATEGORIA_PRED"})

In [12]:
df_act

,observations,end,debt_id,payment_to_bank,inserted_at,bank_reference,CATEGORIA_PRED
0,8:09 a. m. 2/01/2026 VivianR// Se valida con c...,vivian.rodriguez@gobravo.com.co,1898742,3148200.0,2026-01-02 13:10:20,3157363121,CONTRAPROPUESTA
1,8:09 a. m. 2/01/2026 VivianR// Se valida con c...,vivian.rodriguez@gobravo.com.co,1898737,2707100.0,2026-01-02 13:10:53,3157363121,CONTRAPROPUESTA
2,8:09 a. m. 2/01/2026 VivianR// Se valida con c...,vivian.rodriguez@gobravo.com.co,1898741,3137200.0,2026-01-02 13:09:48,3157363121,CONTRAPROPUESTA
3,8:09 a. m. 2/01/2026 VivianR// Se valida con c...,vivian.rodriguez@gobravo.com.co,1898738,2995300.0,2026-01-02 13:11:34,3157363121,CONTRAPROPUESTA
4,8:09 a. m. 2/01/2026 VivianR// Se valida con c...,vivian.rodriguez@gobravo.com.co,1898739,3042600.0,2026-01-02 13:12:05,3157363121,CONTRAPROPUESTA
...,...,...,...,...,...,...,...
6469,12:00 p. m. || Dayana O. || Se escala a contac...,dayana.ojito@gobravo.com.co,1615331,7309966.0,2026-01-15 17:02:51,3216367967,CONTRAPROPUESTA
6470,Vanessa Castillo// 15-01-2026 /12: 04 am se ...,wendy.castillo@gobravo.com.co,1797839,10000000.0,2026-01-15 17:04:07,3105529539,EN_CREDITO
6471,12:00pm//Daniela Sanchez// Me como a la casa d...,maria.sarta@gobravo.com.co,1819744,9000000.0,2026-01-15 17:06:01,3123886715,EN_CREDITO
6472,12:06pm//Daniela Sanchez// se recibe dcto por ...,maria.sarta@gobravo.com.co,1819741,162000000.0,2026-01-15 17:07:53,3123886715,REVISAR


In [13]:
import pandas as pd

# ✅ Compatibilidad ZoneInfo: en Py<3.9 (algunos entornos) usa backports
try:
    from zoneinfo import ZoneInfo
except ImportError:  # pragma: no cover
    from backports.zoneinfo import ZoneInfo  # type: ignore


def construir_timeline_mes(
    df_cartera: pd.DataFrame,
    df_act: pd.DataFrame,
    ref_date=None,
    tz: str = "America/Bogota",
) -> pd.DataFrame:
    """
    Timeline por deuda:
    - Última observación antes del mes
    - Todas las observaciones del mes actual
    Requiere que df_act ya tenga columna 'CATEGORIA_PRED' si quieres usarla.
    """

    # =========================
    # 0) Copias defensivas
    # =========================
    df_c = df_cartera.copy()
    df_a = df_act.copy()

    # =========================
    # 1) Fecha de referencia
    # =========================
    tzinfo = ZoneInfo(tz)

    if ref_date is None:
        ref_date = pd.Timestamp.now(tzinfo)
    else:
        ref_date = pd.Timestamp(ref_date)
        if ref_date.tzinfo is None:
            ref_date = ref_date.tz_localize(tzinfo)
        else:
            ref_date = ref_date.tz_convert(tzinfo)

    month_start = ref_date.replace(day=1, hour=0, minute=0, second=0, microsecond=0)
    next_month_start = month_start + pd.offsets.MonthBegin(1)

    # =========================
    # 2) Normalizar fechas
    # =========================
    df_a["inserted_at"] = pd.to_datetime(df_a["inserted_at"], errors="coerce")

    if getattr(df_a["inserted_at"].dt, "tz", None) is None:
        df_a["inserted_at"] = df_a["inserted_at"].dt.tz_localize(
            tzinfo,
            nonexistent="shift_forward",
            ambiguous="NaT",
        )
    else:
        df_a["inserted_at"] = df_a["inserted_at"].dt.tz_convert(tzinfo)

    # =========================
    # 3) Separar eventos
    # =========================
    mask_mes = (df_a["inserted_at"] >= month_start) & (df_a["inserted_at"] < next_month_start)

    df_mes = df_a.loc[mask_mes].copy()
    df_prev = df_a.loc[df_a["inserted_at"] < month_start].copy()

    # =========================
    # 4) Última antes del mes
    # =========================
    df_prev = df_prev.sort_values(["debt_id", "inserted_at"])
    ultima_prev = df_prev.groupby("debt_id", as_index=False).tail(1)
    ultima_prev["tipo_fila"] = "ultima_antes_mes"

    # =========================
    # 5) Todas las del mes
    # =========================
    df_mes = df_mes.sort_values(["debt_id", "inserted_at"])
    df_mes["tipo_fila"] = "mes_actual"

    # =========================
    # 6) Unir eventos
    # =========================
    eventos = pd.concat([ultima_prev, df_mes], ignore_index=True)

    # Columnas de eventos que quieres traer sí o sí
    cols_eventos = [
        "bank_reference",
        "debt_id",
        "inserted_at",
        "end",
        "payment_to_bank",
        "CATEGORIA_PRED",
        "observations",
        "tipo_fila",
    ]
    for c in cols_eventos:
        if c not in eventos.columns:
            eventos[c] = pd.NA
    eventos = eventos[cols_eventos]

    # =========================
    # 7) Merge con cartera
    # =========================
    df_timeline = df_c.merge(
        eventos,
        left_on=["Referencia", "Id deuda"],
        right_on=["bank_reference", "debt_id"],
        how="left",
    ).drop(columns=["bank_reference", "debt_id"])

    # =========================
    # 8) Garantizar columnas requeridas en el output
    # =========================
    cols_cartera_requeridas = [
        "Referencia",
        "Id deuda",
        "Cedula",
        "Nombre del cliente",
        "Negociador",
        "BANCOS_ESTANDAR",
        "Descuento",
        "D_BRAVO",
        "MORA",
        "Estructurable",
        "Potencial",
        "Meses en el Programa",
        "Tipo de Liquidacion",
        "Bucket",
        "Ahorro total",
        "Ahorro medio",
        "Por cobrar",
        "Potencial Credito",
        "Estado Deuda",
        "sub_estado_deuda",
        "estado_reparadora",
        "sub_estado_reparadora",
        "Mora_estructurado",
        "MORA_CREDITO",
        "Priority_level",
        "ultimo contacto",
        "fecha mensaje"
    ]

    # Si alguna no existe, créala en NA para evitar KeyError
    for c in cols_cartera_requeridas:
        if c not in df_timeline.columns:
            df_timeline[c] = pd.NA

    # Orden sugerido: primero tus columnas de cartera, luego las de eventos (y luego cualquier extra que exista)
    cols_eventos_out = ["inserted_at", "end", "payment_to_bank", "CATEGORIA_PRED", "observations", "tipo_fila"]
    extras = [c for c in df_timeline.columns if c not in (cols_cartera_requeridas + cols_eventos_out)]

    df_timeline = df_timeline[cols_cartera_requeridas + cols_eventos_out + extras]

    # =========================
    # 9) Orden final
    # =========================
    df_timeline = df_timeline.sort_values(
        ["Id deuda", "inserted_at"],
        na_position="first"
    ).reset_index(drop=True)

    return df_timeline


# ===== EJECUCIÓN =====
df_timeline = construir_timeline_mes(df_cartera, df_act)
df_timeline.head(20)

,Referencia,Id deuda,Cedula,Nombre del cliente,Negociador,BANCOS_ESTANDAR,Descuento,D_BRAVO,MORA,Estructurable,...,MORA_CREDITO,Priority_level,ultimo contacto,fecha mensaje,inserted_at,end,payment_to_bank,CATEGORIA_PRED,observations,tipo_fila
0,3012650,111266,52216213,CLAUDIA PATRICIA AVENDANO CALDERON,Negodito,Scotiabank Colpatria,0.7126,9256000.0,120,1,...,No,Prioridad 2,6,NaT,NaT,NaN,NaN,NaN,NaN,NaN
1,3122962933,225561,50868603,Luz Nelly Betancourt Ortega,William Santiago Abril Esguerra,Banco de Occidente,0.5000,2138165.0,55,1,...,No,Prioridad 1,1,2025-12-29 07:31:41,NaT,NaN,NaN,NaN,NaN,NaN
2,3122962933,225562,50868603,Luz Nelly Betancourt Ortega,William Santiago Abril Esguerra,Banco de Occidente,0.5600,3299116.0,55,1,...,No,Prioridad 1,1,2025-12-29 07:31:41,NaT,NaN,NaN,NaN,NaN,NaN
3,3122962933,225563,50868603,Luz Nelly Betancourt Ortega,William Santiago Abril Esguerra,Tuya,0.6715,5959400.0,55,0,...,No,Prioridad 1,1,2025-12-29 07:31:41,NaT,NaN,NaN,NaN,NaN,NaN
4,3196892494,404243,52902886,Diana Marcela Suarez,Dayana Isabel Ojito Ortiz,Banco Popular,0.4800,1921650.0,79,1,...,No,Prioridad 1,6,NaT,NaT,NaN,NaN,NaN,NaN,NaN
5,3183092531,427629,72217923,Faiber Alfonso Lemus Silva,Laura Yineth Torres Moncada,Tuya,0.7126,7481250.0,98,0,...,No,Prioridad 1,6,NaT,NaT,NaN,NaN,NaN,NaN,NaN
6,3003853673,434218,12635912,Alfar de jesus Roca,Cindy Viviana Barrera Buitrago,Scotiabank Colpatria,0.5800,1279300.0,39,1,...,Sí,Prioridad 1,6,NaT,2026-01-09 00:53:45-05:00,massive@resuelvetudeuda.com,5622593.0,ACTUALIZADO_ALIANZAS,Actualizado desde alianzas,mes_actual
7,3003853673,434218,12635912,Alfar de jesus Roca,Cindy Viviana Barrera Buitrago,Scotiabank Colpatria,0.5800,1279300.0,39,1,...,Sí,Prioridad 1,6,NaT,2026-01-13 17:49:46-05:00,cindy.barrera@gobravo.com.co,1279300.0,CONTRAPROPUESTA,"12:54//VivianaB//Se deja en base de contactos,...",mes_actual
8,3003853673,434219,12635912,Alfar de jesus Roca,Cindy Viviana Barrera Buitrago,Scotiabank Colpatria,0.5800,1408000.0,39,1,...,Sí,Prioridad 1,6,NaT,2026-01-09 00:53:45-05:00,massive@resuelvetudeuda.com,5622593.0,ACTUALIZADO_ALIANZAS,Actualizado desde alianzas,mes_actual
9,3003853673,434219,12635912,Alfar de jesus Roca,Cindy Viviana Barrera Buitrago,Scotiabank Colpatria,0.5800,1408000.0,39,1,...,Sí,Prioridad 1,6,NaT,2026-01-13 17:50:15-05:00,cindy.barrera@gobravo.com.co,1408000.0,CONTRAPROPUESTA,"12:54//VivianaB//Se deja en base de contactos,...",mes_actual


In [14]:
df_timeline

,Referencia,Id deuda,Cedula,Nombre del cliente,Negociador,BANCOS_ESTANDAR,Descuento,D_BRAVO,MORA,Estructurable,...,MORA_CREDITO,Priority_level,ultimo contacto,fecha mensaje,inserted_at,end,payment_to_bank,CATEGORIA_PRED,observations,tipo_fila
0,3012650,111266,52216213,CLAUDIA PATRICIA AVENDANO CALDERON,Negodito,Scotiabank Colpatria,0.7126,9256000.0,120,1,...,No,Prioridad 2,6,NaT,NaT,NaN,NaN,NaN,NaN,NaN
1,3122962933,225561,50868603,Luz Nelly Betancourt Ortega,William Santiago Abril Esguerra,Banco de Occidente,0.5000,2138165.0,55,1,...,No,Prioridad 1,1,2025-12-29 07:31:41,NaT,NaN,NaN,NaN,NaN,NaN
2,3122962933,225562,50868603,Luz Nelly Betancourt Ortega,William Santiago Abril Esguerra,Banco de Occidente,0.5600,3299116.0,55,1,...,No,Prioridad 1,1,2025-12-29 07:31:41,NaT,NaN,NaN,NaN,NaN,NaN
3,3122962933,225563,50868603,Luz Nelly Betancourt Ortega,William Santiago Abril Esguerra,Tuya,0.6715,5959400.0,55,0,...,No,Prioridad 1,1,2025-12-29 07:31:41,NaT,NaN,NaN,NaN,NaN,NaN
4,3196892494,404243,52902886,Diana Marcela Suarez,Dayana Isabel Ojito Ortiz,Banco Popular,0.4800,1921650.0,79,1,...,No,Prioridad 1,6,NaT,NaT,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
43036,3225833196,2091518,1019066241,JUAN DAVID LOZANO TRIVINO,Gabriela Saavedra Latorre,Banco Davivienda,0.3637,13695024.0,7,1,...,No,Prioridad 2,0,2026-01-09 09:44:37,NaT,NaN,NaN,NaN,NaN,NaN
43037,3225833196,2091519,1019066241,JUAN DAVID LOZANO TRIVINO,Gabriela Saavedra Latorre,Scotiabank Colpatria,0.5000,50463309.0,7,0,...,No,Prioridad 2,0,2026-01-09 09:44:37,NaT,NaN,NaN,NaN,NaN,NaN
43038,3225833196,2091520,1019066241,JUAN DAVID LOZANO TRIVINO,Gabriela Saavedra Latorre,Bancolombia,0.4700,82022139.0,7,0,...,No,Prioridad 2,0,2026-01-09 09:44:37,NaT,NaN,NaN,NaN,NaN,NaN
43039,3105507630,2091523,79381039,JUAN CARLOS PACHECO CONTRERAS,Julian Andres Ossa Bohorquez,Bancolombia,0.4700,130935011.0,7,0,...,No,Prioridad 2,0,2026-01-09 10:51:57,NaT,NaN,NaN,NaN,NaN,NaN


In [15]:
# -*- coding: utf-8 -*-
import os
import json
import pandas as pd
import gspread
from google.oauth2.service_account import Credentials

# =========================
# CONFIG
# =========================
SPREADSHEET_ID = "1H3sYEtkeu47POnu8xZMaMtID1Vj53YIcWblWeZ8d0rc"
GID = "1033250632"

# =========================
# 1) Leer MI_JSON según entorno (Colab vs GitHub/local)
# =========================
def get_mi_json():
    # --- Colab ---
    try:
        from google.colab import userdata  # solo existe en Colab
        mi_json = userdata.get("MI_JSON")
        if not mi_json:
            raise RuntimeError("MI_JSON no encontrado en Colab userdata.")
        print("Entorno detectado: Google Colab")
        return mi_json
    except Exception:
        # --- GitHub Actions / local ---
        mi_json = os.environ.get("MI_JSON")
        if not mi_json:
            raise RuntimeError("MI_JSON no encontrado como variable de entorno (GitHub/local).")
        if os.environ.get("GITHUB_ACTIONS") == "true":
            print("Entorno detectado: GitHub Actions")
        else:
            print("Entorno detectado: Local")
        return mi_json

mi_json = get_mi_json()
info = json.loads(mi_json)

SCOPES = [
    "https://www.googleapis.com/auth/spreadsheets.readonly",
    "https://www.googleapis.com/auth/drive.readonly",
]
creds = Credentials.from_service_account_info(info, scopes=SCOPES)
gc = gspread.authorize(creds)

sh = gc.open_by_key(SPREADSHEET_ID)

# =========================
# 2) Resolver el nombre de la hoja a partir del gid
# =========================
meta = sh.fetch_sheet_metadata()
sheet_title = None
for s in meta.get("sheets", []):
    props = s.get("properties", {})
    if str(props.get("sheetId")) == str(GID):
        sheet_title = props.get("title")
        break

if not sheet_title:
    raise ValueError(f"No encontré ninguna hoja con gid={GID}. Revisa el link.")

ws = sh.worksheet(sheet_title)

# =========================
# 3) Leer a DataFrame
# =========================
values = ws.get_all_values()

if not values or len(values) < 2:
    df_liq = pd.DataFrame()
else:
    headers = values[0]
    rows = values[1:]
    df_liq = pd.DataFrame(rows, columns=headers)

print("✅ Hoja leída:", sheet_title)
print("Shape:", df_liq.shape)
print(df_liq.head(20))

Entorno detectado: GitHub Actions


✅ Hoja leída: BD del mes
Shape: (2658, 27)
   Pago a banco Deuda Resuelve  Fecha de Liquidación Comisión  \
0        500000         5000100            6/01/2026  1222515   
1       1000000         7225050            6/01/2026            
2         78000           40000            5/01/2026   815466   
3        128000          196441            5/01/2026            
4        208000          200000            5/01/2026            
5         60000          600000            5/01/2026            
6         70000          720000            5/01/2026            
7         87000          900000            5/01/2026            
8        335000         3700000            5/01/2026            
9       3000000        17234700            5/01/2026  2135205   
10      2154000         9620100            6/01/2026  1493220   
11       180100          674100            6/01/2026    98800   
12       200000         1600200            6/01/2026   280040   
13      5249800        26663700            6/01

In [16]:
df_liq

,Pago a banco,Deuda Resuelve,Fecha de Liquidación,Comisión,Negociador,Portafolio,Deuda Berex,Referencia,Nombre cliente,Banco,...,Tipo EST,Dia operativo,Check,Casa de cobro,Lider,,,,,
0,500000,5000100,6/01/2026,1222515,William Santiago Abril Esguerra,SI,1826700,3156617917,Diego Fernando Gaitan Quintero,Davivienda,...,comisión,2,,0,Juan Jose Hernandez Galvis,TRUE,"27,17%",,1681103,comisión
1,1000000,7225050,6/01/2026,,William Santiago Abril Esguerra,SI,1826701,3156617917,Diego Fernando Gaitan Quintero,Davivienda,...,,2,,0,Juan Jose Hernandez Galvis,FALSE,"0,00%",,1726199,comisión
2,78000,40000,5/01/2026,815466,Vivian Caterin Rodriguez Verano,SI,1892087,3104999127,Edison Fernando Hincapie Arcila,Bancolombia,...,comisión,1,,0,Natalia Valentina Castro Jimenez,TRUE,"-2145,96%",,1773378,comisión
3,128000,196441,5/01/2026,,Vivian Caterin Rodriguez Verano,SI,1892088,3104999127,Edison Fernando Hincapie Arcila,Bancolombia,...,,1,,0,Natalia Valentina Castro Jimenez,FALSE,"0,00%",,1203220,comisión
4,208000,200000,5/01/2026,,Vivian Caterin Rodriguez Verano,SI,1892089,3104999127,Edison Fernando Hincapie Arcila,Bancolombia,...,,1,,0,Natalia Valentina Castro Jimenez,FALSE,"0,00%",,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2653,,,,,,,,,,,...,,,,0,,,,,,
2654,,,,,,,,,,,...,,,,0,,,,,,
2655,,,,,,,,,,,...,,,,0,,,,,,
2656,,,,,,,,,,,...,,,,0,,,,,,


In [17]:
pip install fuzzywuzzy[speedup]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3/3 [python-levenshtein]


Note: you may need to restart the kernel to use updated packages.


In [18]:
from rapidfuzz import process, fuzz
import pandas as pd
import re

nombres_correctos = [
    'Bancolombia', 'Banco Falabella', 'Banco de Bogotá', 'Banco Davivienda',
    'Scotiabank Colpatria', 'BBVA Colombia', 'SisteCredito', 'Banco AV Villas',
    'Banco de Occidente', 'Alkomprar', 'Tuya', 'Codensa', 'Itaú',
    'Serfinanza', 'Credivalores', 'Banco Popular', 'Rappipay',
    'Banco Finandina', 'Banco Caja Social', 'Covinoc', 'Bancoomeva',
    'Rapicredit', 'Credijamar', 'Flamingo', 'Zinobe', 'Bancamia',
    'Refinancia', 'Compensar', 'Pichincha', 'Colsubsidio', 'Mundo Mujer',
    'Agaval', 'Sistemcobro', 'Serlefin', 'Rappicard', 'Fincomercio',
    'GRUPO JURIDICO DEUDU', 'AECSA', 'Sufi', 'Comultrasan', 'Fundación',
    'Cobrando', 'Aslegal', 'Coltefinanciera', 'Reestructura', 'Nu',
    'Juancho te Presta', 'Muebles', 'JOHN', 'GNB Sudameris', 'Confiar',
    'Baninca', 'Systemgroup', 'Efecty'
]

# Bancos que se usan con el prefijo "banco" completo como referencia
bancos_excepciones = ['Banco de Bogotá', 'Bancolombia', 'Bancoomeva']

# ==============================
# 2. Sinónimos / patrones por banco (texto original, sin limpiar)
# ==============================
PATRONES_POR_BANCO = {
    'Bancolombia': [
        'bancolombia', 'contento bancolombia', 'qnt bancolombia', 'sufi'
    ],
    'Banco Davivienda': [
        'davivienda', 'intercredito davivienda', 'gestiones profesionales davivienda',
        'inversionistas estratégicos davivienda', 'inversionistas estrategicos davivienda',
        'management davivienda', 'davivienda cobrado sas', 'davivienda cobrando sas',
        'deudu davivienda', 'qnt davivienda'
    ],
    'BBVA Colombia': [
        'bbva', 'cobrando bbva', 'beta bbva', 'cobranzas beta origen: bbva',
        'aecsa bbva', 'grupo juridico bbva', 'grupo jurídico bbva', 'qnt bbva'
    ],
    'Banco Falabella': [
        'falabella', 'bfalabella', 'bancofalab', 'bancofalab citisumma',
        'banco falabella casa de cobro', 'eyc falabella',
        'cobrando falabella', 'acr logros f ori falabella',
        'deudu falabella', 'deudo falabella', 'citisumma falabella',
        'logros factoring falabella'
    ],
    'Tuya': [
        ' tuya', 'viva-tuya', 'viva tuya', 'qnt tuya', 'aecsa tuya',
        'tuya contacto soluciones', 'tuya s.a contactosol', 'tuya s a contactosol',
        'tuya contacto soluciones', 'qnt tuya',
        'éxito', 'exito', 'carulla', 'Alkosto', 'Corbeta'
    ],
    'Scotiabank Colpatria': [
        'scotiabank', 'skotiabank', 'colpatria', 'peruzzi skotiabank colpatria',
        'serlefin colpatria', 'adamantine scotiabank', 'gc andino colpatria',
        'scotiabank citibank', 'qnt colpatria', 'crc colpatria',
        'grupo consulto colpatria', 'grupo consultor andino colpatria',
        'gr.consulto colpatria', 'Codensa'
    ],
    'Banco de Bogotá': [
        'banco de bogota', 'banco de bogotá', 'qnt bogota', 'qnt bogotá',
        'crear país banco de bogotá', 'crear pais banco de bogota'
    ],
    'Banco de Occidente': [
        'banco de occidente', 'qnt banco de occidente', 'deudu-banco de occidente',
        'deudu banco de occidente'
    ],
    'Banco Popular': [
        'banco popular', 'banco popular casa de cobro', 'banco popular contactosol',
        'banco popular contactosolsas', 'banco popular citisumma',
        'banco popular-adcore', 'deudu banco popular', 'peruzzicol bcopopular'
    ],
    'Banco AV Villas': [
        'av villas', 'banco av villas', 'grupo consultor andino av villas',
        'grupo juridico av villas', 'grupo jurídico av villas',
        'ae csa av villas', 'aecsa av villas', 'crear país banco av villas',
        'crear pais banco av villas', 'deudu av villas', 'qnt av villas'
    ],
    'Banco Caja Social': [
        'banco caja social', 'caja social', 'pic caja social'
    ],
    'Bancoomeva': [
        'banco coomeva', 'bancoomeva', 'coomeva'
    ],
    'Bancamia': [
        'bancamia', 'bancamia s.a', 'bancamia s a'
    ],
    'Mundo Mujer': [
        'banco mundo mujer', 'fundacion de la mujer', 'fundación de la mujer'
    ],
    'SisteCredito': [
        'sistecredito', 'sistecrédito'
    ],
    'Covinoc': ['covinoc'],
    'Compensar': ['compensar'],
    'Pichincha': ['pichincha', 'pichincha educativo'],
    'Agaval': ['agaval'],
    'Banco Finandina': [
        'finandina', 'finandina incomercio', 'finandina incomercio'
    ],
    'Fincomercio': ['fincomercio'],
    'Serfinanza': [
        'serfinanza', 'serfinansa', 'serfinanza contactosol',
        'contacto solucion serfinanza', 'contacto solución serfinanza'
    ],
    'Credijamar': [
        'credijamar', 'muebles jamar'
    ],
    'Juancho te Presta': ['juancho te presta'],
    'Rapicredit': ['rapicredit'],
    'Zinobe': ['zinobe'],
    'Coltefinanciera': ['coltefinanciera'],
    'Sistemcobro': ['sistemcobro'],
    'Systemgroup': ['systemgroup'],
    'Baninca': ['baninca'],
    'GNB Sudameris': ['gnb sudameris'],
    'Confiar': ['confiar'],
    'AECSA': ['aecsa'],
    'Comultrasan': ['comultrasan'],
    'Nu': ['nu bank', 'nubank', 'logros factoring nubank'],
    'Rappi':['Rappicard', 'Rappipay'],
    'Lulo Bank': ['LuloBank', 'Lulo Banck'],
    'Banco union': ['QNT GIROS&FINANZAS']

}

# ==============================
# 3. Alias manuales sobre texto LIMPIO
# ==============================
alias_manuales = {
    'bfalabella': 'Banco Falabella',
    'contactosol': 'Banco Falabella',
    'qnt itau': 'Itaú',
    'itau helm': 'Itaú',
    'itau corpbanca': 'Itaú',
    'viva tuya': 'Tuya',
    'carulla': 'Tuya',
    'sufi': 'Bancolombia',
    'adamantine scotiabank': 'Scotiabank Colpatria',
    'gc andino colpatria': 'Scotiabank Colpatria',
    'beta bbva': 'BBVA Colombia',
    'banco popular': 'Banco Popular',
    'qnt bogota': 'Banco de Bogotá',
    'banco caja social': 'Banco Caja Social',
    'banco av villas': 'Banco AV Villas',
    'banco davivienda': 'Banco Davivienda',
    'bancofalab citisumma': 'Banco Falabella',
    'exito': 'Tuya',
    'éxito': 'Tuya',
    'Alkosto': 'Tuya',
    'Corbeta': 'Tuya',
    'Rappipay': 'Rappi',
    'Rappicard': 'Rappi',
    'Lulo Banck': 'Lulo Bank',
    'LuloBank': 'Lulo Bank',
    'QNT GIROS&FINANZAS': 'Banco Union',
    'Codensa': 'Scotiabank Colpatria'


}

# ==============================
# 4. Función de limpieza
# ==============================
def limpiar_texto(texto):
    """
    Limpia y normaliza el texto: minúsculas, elimina caracteres no alfabéticos
    (excepto acentos y ñ/ü), elimina palabras irrelevantes y quita espacios dobles.
    """
    texto = str(texto).lower()

    # Dejar solo letras, acentos, ñ, ü y espacios
    texto = re.sub(r'[^a-záéíóúñü\s]', ' ', texto)

    # Eliminar palabras "de relleno" frecuentes en reparadoras / BPO
    texto = re.sub(
        r'\b('
        r'grupo|juridico|jurídico|sas|sa|s a|ltda|suma|financiera|'
        r'contactosol|contacto|solucion|soluciones|citisumma|'
        r'cobrando|cobranzas|adcore|logros|factoring|origen|origem|'
        r'gestiones|gestion|profesionales|bpo|inversionistas|'
        r'estrategicos|estratégicos|casa|de|cobro|servicios|'
        r'creditos|credito|abogados|asociados|'
        r'outsourcing|risk|patrimonio|autonomo|autónomo|central|'
        r'inversiones|valora|punto|com|puntocom|activos|'
        r'recuperacion|recuperación|financiera|financiero|'
        r'asesores|asociados|gest|prof|eyc|gca|summa'
        r')\b',
        '',
        texto
    )

    # Espacios múltiples -> uno solo
    texto = re.sub(r'\s+', ' ', texto).strip()
    return texto

# ==============================
# 5. Claves de referencia para fuzzy
# ==============================
claves_referencia = {
    (n.lower() if n in bancos_excepciones else re.sub(r'^banco\s*', '', n.lower())): n
    for n in nombres_correctos
}

# ==============================
# 6. Mapeo por patrón (texto original)
# ==============================
def mapear_por_patron(nombre_incorrecto):
    """
    Intenta identificar el banco a partir del texto ORIGINAL (sin limpiar),
    buscando los patrones definidos en PATRONES_POR_BANCO.
    """
    if pd.isna(nombre_incorrecto):
        return None

    texto = str(nombre_incorrecto).lower()

    for banco_estandar, patrones in PATRONES_POR_BANCO.items():
        for patron in patrones:
            if patron in texto:
                return banco_estandar

    return None

# ==============================
# 7. Función principal de corrección
# ==============================
def corregir_nombre(nombre_incorrecto):
    """
    Corrige un nombre de banco usando 3 capas:
    1. Mapeo por patrón sobre el texto original.
    2. Limpieza de texto + alias manuales.
    3. Fuzzy matching contra claves_referencia.
    """
    # Si es NaN/None, lo dejamos igual
    if pd.isna(nombre_incorrecto):
        return nombre_incorrecto

    # 1. Intentar primero con patrones evidentes en el texto original
    banco_patron = mapear_por_patron(nombre_incorrecto)
    if banco_patron is not None:
        return banco_patron

    # 2. Limpiar texto
    limpio = limpiar_texto(nombre_incorrecto)

    # Si después de limpiar no queda nada, devolvemos el original
    if limpio == '':
        return nombre_incorrecto

    # 3. Alias manuales sobre texto limpio
    for alias, banco_estandar in alias_manuales.items():
        if alias in limpio:
            return banco_estandar

    # 4. Fuzzy matching con RapidFuzz
    mejor_match, score, _ = process.extractOne(
        limpio,
        claves_referencia.keys(),
        scorer=fuzz.token_set_ratio
    )

    # Si la similitud es alta, usamos el banco estándar; si no, dejamos el texto original
    return claves_referencia[mejor_match] if score > 70 else nombre_incorrecto

# ==============================
# 8. Aplicar al DataFrame
# ==============================

# Crear nueva columna estandarizada en reparadoras_df
df_liq['BANCOS_ESTANDAR'] = df_liq['Banco'].apply(corregir_nombre)

In [19]:
import pandas as pd
import numpy as np

# ✅ ZoneInfo compatible con Colab y GitHub (Py < 3.9)
try:
    from zoneinfo import ZoneInfo
except ImportError:  # pragma: no cover
    from backports.zoneinfo import ZoneInfo  # type: ignore


def _to_float_money(x):
    if pd.isna(x):
        return np.nan
    s = str(x).strip()
    if s == "":
        return np.nan

    s = s.replace("$", "").replace("COP", "").replace("cop", "").replace(" ", "")

    if "," in s and "." in s:
        if s.rfind(",") > s.rfind("."):
            s = s.replace(".", "")
            s = s.replace(",", ".")
        else:
            s = s.replace(",", "")
    else:
        if "," in s:
            if s.count(",") > 1:
                s = s.replace(",", "")
            else:
                tail = s.split(",")[-1]
                s = s.replace(",", ".") if len(tail) in (1, 2) else s.replace(",", "")
        if "." in s:
            if s.count(".") > 1:
                s = s.replace(".", "")
            else:
                tail = s.split(".")[-1]
                if len(tail) not in (1, 2):
                    s = s.replace(".", "")

    try:
        return float(s)
    except Exception:
        return np.nan


def _to_bogota_datetime_ddmmyyyy(series, tz="America/Bogota"):
    tzinfo = ZoneInfo(tz)
    dt = pd.to_datetime(series, errors="coerce", dayfirst=True)
    if getattr(dt.dt, "tz", None) is None:
        dt = dt.dt.tz_localize(tzinfo, nonexistent="shift_forward", ambiguous="NaT")
    else:
        dt = dt.dt.tz_convert(tzinfo)
    return dt


def _modo(s: pd.Series):
    s = s.dropna().astype(str)
    if s.empty:
        return np.nan
    return s.value_counts().index[0]


def agregar_liquidaciones_al_timeline_con_fallback(
    df_timeline: pd.DataFrame,
    df_liq: pd.DataFrame,
) -> pd.DataFrame:
    df_tl = df_timeline.copy()
    df_l = df_liq.copy()

    # asegurar columnas nuevas
    for c in ["Negociador liquidacion", "Por?"]:
        if c not in df_tl.columns:
            df_tl[c] = np.nan

    # normalizar llaves
    df_l["Id deuda"] = pd.to_numeric(df_l["Deuda Berex"], errors="coerce").astype("Int64")
    df_l["Referencia"] = pd.to_numeric(df_l["Referencia"], errors="coerce").astype("Int64")

    # fecha dd/mm/yyyy -> datetime Bogotá
    df_l["inserted_at"] = _to_bogota_datetime_ddmmyyyy(df_l["Fecha de Liquidación"])

    # pago a banco float
    df_l["payment_to_bank"] = df_l["Pago a banco"].apply(_to_float_money).astype(float)

    # mapas desde timeline para fallback
    base_by_debt = (
        df_tl[
            ["Id deuda", "Referencia", "BANCOS_ESTANDAR", "Descuento", "D_BRAVO", "Tipo de Liquidacion"]
        ]
        .dropna(subset=["Id deuda"])
        .drop_duplicates(subset=["Id deuda"])
        .set_index("Id deuda")
    )

    map_ref_by_debt = base_by_debt["Referencia"].to_dict()
    map_banco_by_debt = base_by_debt["BANCOS_ESTANDAR"].to_dict()
    map_desc_by_debt = base_by_debt["Descuento"].to_dict()
    map_deuda_by_debt = base_by_debt["D_BRAVO"].to_dict()
    map_tipol_by_debt = base_by_debt["Tipo de Liquidacion"].to_dict()

    # mapa negociador por referencia
    map_neg_ref = (
        df_tl[["Referencia", "Negociador"]]
        .dropna(subset=["Referencia", "Negociador"])
        .groupby("Referencia")["Negociador"]
        .apply(_modo)
        .to_dict()
    )

    banco_liq = df_l["BANCO_ESTANDAR"] if "BANCO_ESTANDAR" in df_l.columns else pd.Series(np.nan, index=df_l.index)
    deuda_res_liq = df_l["D_BRAVO"] if "D_BRAVO" in df_l.columns else pd.Series(np.nan, index=df_l.index)

    if "descuento" in df_l.columns:
        desc_liq = pd.to_numeric(df_l["descuento"], errors="coerce")
    else:
        desc_liq = pd.Series(np.nan, index=df_l.index)

    tipo_liq_liq = df_l["Tipo de liquidacion"].astype(object)

    filas_liq = pd.DataFrame({
        "Referencia": df_l["Referencia"],
        "Id deuda": df_l["Id deuda"],
        "Negociador": np.nan,
        "BANCOS_ESTANDAR": banco_liq.astype(object),
        "Descuento": desc_liq.astype(float),
        "D_BRAVO": deuda_res_liq,
        "Tipo de Liquidacion": tipo_liq_liq,
        "inserted_at": df_l["inserted_at"],
        "end": np.nan,
        "payment_to_bank": df_l["payment_to_bank"],
        "CATEGORIA_PRED": "Liquidado",
        "observations": np.nan,
        "tipo_fila": "Liquidación",
        "Negociador liquidacion": df_l["Negociador"].astype(str),
        "Por?": df_l["Tipo de liquidacion"].astype(str),
    })

    filas_liq = (
        filas_liq.dropna(subset=["Id deuda"])
        .sort_values(["Id deuda", "inserted_at"])
        .groupby("Id deuda", as_index=False)
        .tail(1)
    )

    filas_liq["Referencia"] = filas_liq["Referencia"].fillna(filas_liq["Id deuda"].map(map_ref_by_debt))
    filas_liq["Negociador"] = filas_liq["Referencia"].map(map_neg_ref)

    filas_liq["BANCOS_ESTANDAR"] = filas_liq["BANCOS_ESTANDAR"].fillna(filas_liq["Id deuda"].map(map_banco_by_debt))
    filas_liq["Descuento"] = filas_liq["Descuento"].fillna(filas_liq["Id deuda"].map(map_desc_by_debt))

    filas_liq["D_BRAVO"] = pd.to_numeric(filas_liq["D_BRAVO"], errors="coerce")
    filas_liq["D_BRAVO"] = filas_liq["D_BRAVO"].fillna(filas_liq["Id deuda"].map(map_deuda_by_debt))

    filas_liq["Tipo de Liquidacion"] = filas_liq["Tipo de Liquidacion"].replace("", np.nan)
    filas_liq["Tipo de Liquidacion"] = filas_liq["Tipo de Liquidacion"].fillna(
        filas_liq["Id deuda"].map(map_tipol_by_debt)
    )

    filas_liq["Referencia"] = filas_liq["Referencia"].astype("Int64")
    filas_liq["Id deuda"] = filas_liq["Id deuda"].astype("Int64")
    filas_liq["Descuento"] = pd.to_numeric(filas_liq["Descuento"], errors="coerce").astype(float)
    filas_liq["D_BRAVO"] = pd.to_numeric(filas_liq["D_BRAVO"], errors="coerce").astype(float)
    filas_liq["payment_to_bank"] = pd.to_numeric(filas_liq["payment_to_bank"], errors="coerce").astype(float)
    filas_liq["inserted_at"] = _to_bogota_datetime_ddmmyyyy(filas_liq["inserted_at"])

    for c in df_tl.columns:
        if c not in filas_liq.columns:
            filas_liq[c] = np.nan
    for c in filas_liq.columns:
        if c not in df_tl.columns:
            df_tl[c] = np.nan

    filas_liq = filas_liq[df_tl.columns]

    out = (
        pd.concat([df_tl, filas_liq], ignore_index=True)
        .sort_values(["Id deuda", "inserted_at"], na_position="first")
        .reset_index(drop=True)
    )

    return out


# ===== USO =====
df_timeline_final = agregar_liquidaciones_al_timeline_con_fallback(df_timeline, df_liq)
df_timeline_final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 43178 entries, 0 to 43177
Data columns (total 35 columns):
 #   Column                  Non-Null Count  Dtype                         
---  ------                  --------------  -----                         
 0   Referencia              43178 non-null  Int64                         
 1   Id deuda                43178 non-null  Int64                         
 2   Cedula                  43041 non-null  float64                       
 3   Nombre del cliente      43041 non-null  object                        
 4   Negociador              43157 non-null  object                        
 5   BANCOS_ESTANDAR         43157 non-null  object                        
 6   Descuento               38197 non-null  float64                       
 7   D_BRAVO                 43157 non-null  float64                       
 8   MORA                    43041 non-null  float64                       
 9   Estructurable           43041 non-null  float64   

/tmp/ipykernel_2204/4040414486.py:177: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  pd.concat([df_tl, filas_liq], ignore_index=True)


In [20]:
import numpy as np

df_timeline = df_timeline_final.copy()

# -------------------------
# 1) Pago banco esperado
# -------------------------
df_timeline["Pago_banco_esperado"] = (
    df_timeline["D_BRAVO"]
    - (df_timeline["D_BRAVO"] * df_timeline["Descuento"])
)

# -------------------------
# 2) Ingreso esperado (CE = 0.15)
# -------------------------
CE = 0.15

df_timeline["Ingreso_esperado"] = np.maximum(
    0,
    (df_timeline["D_BRAVO"] - df_timeline["Pago_banco_esperado"]) * 1.19 * CE
)

# -------------------------
# 3) Mover columna al lado de D_BRAVO
# -------------------------
cols = df_timeline.columns.tolist()
idx = cols.index("D_BRAVO")

# quitamos y reinsertamos
cols.remove("Ingreso_esperado")
cols.insert(idx + 1, "Ingreso_esperado")

df_timeline = df_timeline[cols]

df_timeline[[
    "D_BRAVO",
    "Descuento",
    "Pago_banco_esperado",
    "Ingreso_esperado"
]].head()

,D_BRAVO,Descuento,Pago_banco_esperado,Ingreso_esperado
0,9256000.0,0.7126,2660174.40,1.177355e+06
1,2138165.0,0.5000,1069082.50,1.908312e+05
2,3299116.0,0.5600,1451611.04,3.297796e+05
3,5959400.0,0.6715,1957662.90,7.143101e+05
4,1921650.0,0.4800,999258.00,1.646470e+05


In [21]:
df_timeline

,Referencia,Id deuda,Cedula,Nombre del cliente,Negociador,BANCOS_ESTANDAR,Descuento,D_BRAVO,Ingreso_esperado,MORA,...,fecha mensaje,inserted_at,end,payment_to_bank,CATEGORIA_PRED,observations,tipo_fila,Negociador liquidacion,Por?,Pago_banco_esperado
0,3012650,111266,5.221621e+07,CLAUDIA PATRICIA AVENDANO CALDERON,Negodito,Scotiabank Colpatria,0.7126,9256000.0,1.177355e+06,120.0,...,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.660174e+06
1,3122962933,225561,5.086860e+07,Luz Nelly Betancourt Ortega,William Santiago Abril Esguerra,Banco de Occidente,0.5000,2138165.0,1.908312e+05,55.0,...,2025-12-29 07:31:41,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.069082e+06
2,3122962933,225562,5.086860e+07,Luz Nelly Betancourt Ortega,William Santiago Abril Esguerra,Banco de Occidente,0.5600,3299116.0,3.297796e+05,55.0,...,2025-12-29 07:31:41,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.451611e+06
3,3122962933,225563,5.086860e+07,Luz Nelly Betancourt Ortega,William Santiago Abril Esguerra,Tuya,0.6715,5959400.0,7.143101e+05,55.0,...,2025-12-29 07:31:41,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.957663e+06
4,3196892494,404243,5.290289e+07,Diana Marcela Suarez,Dayana Isabel Ojito Ortiz,Banco Popular,0.4800,1921650.0,1.646470e+05,79.0,...,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9.992580e+05
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
43173,3225833196,2091518,1.019066e+09,JUAN DAVID LOZANO TRIVINO,Gabriela Saavedra Latorre,Banco Davivienda,0.3637,13695024.0,8.890871e+05,7.0,...,2026-01-09 09:44:37,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8.714144e+06
43174,3225833196,2091519,1.019066e+09,JUAN DAVID LOZANO TRIVINO,Gabriela Saavedra Latorre,Scotiabank Colpatria,0.5000,50463309.0,4.503850e+06,7.0,...,2026-01-09 09:44:37,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.523165e+07
43175,3225833196,2091520,1.019066e+09,JUAN DAVID LOZANO TRIVINO,Gabriela Saavedra Latorre,Bancolombia,0.4700,82022139.0,6.881247e+06,7.0,...,2026-01-09 09:44:37,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.347173e+07
43176,3105507630,2091523,7.938104e+07,JUAN CARLOS PACHECO CONTRERAS,Julian Andres Ossa Bohorquez,Bancolombia,0.4700,130935011.0,1.098479e+07,7.0,...,2026-01-09 10:51:57,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6.939556e+07


In [22]:
import pandas as pd
import numpy as np
import re
import unicodedata

df_liq_aux = df_liq.copy()

def _norm_col(x: str) -> str:
    # normaliza: lower, quita tildes, colapsa espacios
    x = str(x).replace("\u00a0", " ")
    x = re.sub(r"\s+", " ", x).strip().lower()
    x = "".join(ch for ch in unicodedata.normalize("NFD", x) if unicodedata.category(ch) != "Mn")
    return x

# --- 1) Encontrar el nombre REAL de columnas (robusto)
cols_norm = {c: _norm_col(c) for c in df_liq_aux.columns}

# deuda (lo que tú quieres para D_BRAVO) = "Deuda Resuelve"
col_deuda = next((c for c, cn in cols_norm.items() if cn in ("d_bravo", "deuda resuelve", "deuda_resuelve")), None)

# banco
col_banco = next((c for c, cn in cols_norm.items() if cn in ("bancos_estandar", "banco estandar", "banco_estandar")), None)

# id berex
col_berex = next((c for c, cn in cols_norm.items() if cn in ("deuda berex", "deuda_berex", "id deuda berex", "id_deuda_berex")), None)

print("Columna deuda detectada:", col_deuda)
print("Columna banco detectada:", col_banco)
print("Columna berex detectada:", col_berex)

if col_deuda is None:
    raise KeyError("No encontré la columna de deuda (esperaba 'Deuda Resuelve' o 'D_BRAVO').")
if col_banco is None:
    raise KeyError("No encontré la columna 'BANCOS_ESTANDAR' (ni variantes).")
if col_berex is None:
    raise KeyError("No encontré la columna 'Deuda Berex' (ni variantes).")

# --- 2) Construir Id deuda desde Deuda Berex
df_liq_aux["Id deuda"] = pd.to_numeric(df_liq_aux[col_berex], errors="coerce").astype("Int64")

# --- 3) Parse D_BRAVO a float (robusto)
s = df_liq_aux[col_deuda].astype(str).str.strip()

# limpiar símbolos y separar miles/decimales típico de COP
s = s.str.replace(r"[^\d,\.]", "", regex=True)

# caso típico: "45.488.000" -> quitar puntos miles
# y "45,488,000" -> quitar comas miles
# y si viene con decimal, lo intentamos preservar
# estrategia: si tiene ambos, asumimos decimal el último separador
def _parse_money_str(x):
    if x in ("", "nan", "None"):
        return np.nan
    if "," in x and "." in x:
        # el separador que aparezca más a la derecha es decimal
        if x.rfind(",") > x.rfind("."):
            x = x.replace(".", "")
            x = x.replace(",", ".")
        else:
            x = x.replace(",", "")
    else:
        # si solo tiene comas: puede ser miles o decimal
        if "," in x:
            parts = x.split(",")
            # si último bloque tiene 1-2 dígitos, decimal
            if len(parts[-1]) in (1,2):
                x = x.replace(".", "")
                x = x.replace(",", ".")
            else:
                x = x.replace(",", "")
        # si solo tiene puntos: miles o decimal
        if "." in x:
            parts = x.split(".")
            if len(parts[-1]) not in (1,2):  # no parece decimal
                x = x.replace(".", "")
    try:
        return float(x)
    except:
        return np.nan

df_liq_aux["Deuda_Resuelve_num"] = s.map(_parse_money_str)

# --- 4) Mapas por Id deuda
map_banco_liq = (
    df_liq_aux.dropna(subset=["Id deuda", col_banco])
             .drop_duplicates(subset=["Id deuda"])
             .set_index("Id deuda")[col_banco]
)

map_deuda_liq = (
    df_liq_aux.dropna(subset=["Id deuda", "Deuda_Resuelve_num"])
             .drop_duplicates(subset=["Id deuda"])
             .set_index("Id deuda")["Deuda_Resuelve_num"]
)

# --- 5) Rellenar SOLO filas Liquidación y SOLO si está nulo
df = df_timeline.copy()
mask_liq = df["tipo_fila"].astype("string").eq("Liquidación")

df.loc[mask_liq, "BANCOS_ESTANDAR"] = (
    df.loc[mask_liq, "BANCOS_ESTANDAR"]
      .fillna(df.loc[mask_liq, "Id deuda"].map(map_banco_liq))
)

df.loc[mask_liq, "D_BRAVO"] = (
    df.loc[mask_liq, "D_BRAVO"]
      .fillna(df.loc[mask_liq, "Id deuda"].map(map_deuda_liq))
)

df["D_BRAVO"] = pd.to_numeric(df["D_BRAVO"], errors="coerce").astype(float)

df_timeline_final = df

# --- 6) Chequeo
m = df_timeline_final["tipo_fila"].astype("string").eq("Liquidación")
print("Liquidación sin BANCOS_ESTANDAR:", df_timeline_final.loc[m, "BANCOS_ESTANDAR"].isna().sum())
print("Liquidación sin D_BRAVO:", df_timeline_final.loc[m, "D_BRAVO"].isna().sum())

Columna deuda detectada: Deuda Resuelve 
Columna banco detectada: BANCOS_ESTANDAR
Columna berex detectada: Deuda Berex
Liquidación sin BANCOS_ESTANDAR: 0
Liquidación sin D_BRAVO: 0


In [23]:
df_timeline_final

,Referencia,Id deuda,Cedula,Nombre del cliente,Negociador,BANCOS_ESTANDAR,Descuento,D_BRAVO,Ingreso_esperado,MORA,...,fecha mensaje,inserted_at,end,payment_to_bank,CATEGORIA_PRED,observations,tipo_fila,Negociador liquidacion,Por?,Pago_banco_esperado
0,3012650,111266,5.221621e+07,CLAUDIA PATRICIA AVENDANO CALDERON,Negodito,Scotiabank Colpatria,0.7126,9256000.0,1.177355e+06,120.0,...,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.660174e+06
1,3122962933,225561,5.086860e+07,Luz Nelly Betancourt Ortega,William Santiago Abril Esguerra,Banco de Occidente,0.5000,2138165.0,1.908312e+05,55.0,...,2025-12-29 07:31:41,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.069082e+06
2,3122962933,225562,5.086860e+07,Luz Nelly Betancourt Ortega,William Santiago Abril Esguerra,Banco de Occidente,0.5600,3299116.0,3.297796e+05,55.0,...,2025-12-29 07:31:41,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.451611e+06
3,3122962933,225563,5.086860e+07,Luz Nelly Betancourt Ortega,William Santiago Abril Esguerra,Tuya,0.6715,5959400.0,7.143101e+05,55.0,...,2025-12-29 07:31:41,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.957663e+06
4,3196892494,404243,5.290289e+07,Diana Marcela Suarez,Dayana Isabel Ojito Ortiz,Banco Popular,0.4800,1921650.0,1.646470e+05,79.0,...,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9.992580e+05
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
43173,3225833196,2091518,1.019066e+09,JUAN DAVID LOZANO TRIVINO,Gabriela Saavedra Latorre,Banco Davivienda,0.3637,13695024.0,8.890871e+05,7.0,...,2026-01-09 09:44:37,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8.714144e+06
43174,3225833196,2091519,1.019066e+09,JUAN DAVID LOZANO TRIVINO,Gabriela Saavedra Latorre,Scotiabank Colpatria,0.5000,50463309.0,4.503850e+06,7.0,...,2026-01-09 09:44:37,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.523165e+07
43175,3225833196,2091520,1.019066e+09,JUAN DAVID LOZANO TRIVINO,Gabriela Saavedra Latorre,Bancolombia,0.4700,82022139.0,6.881247e+06,7.0,...,2026-01-09 09:44:37,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.347173e+07
43176,3105507630,2091523,7.938104e+07,JUAN CARLOS PACHECO CONTRERAS,Julian Andres Ossa Bohorquez,Bancolombia,0.4700,130935011.0,1.098479e+07,7.0,...,2026-01-09 10:51:57,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6.939556e+07


In [24]:
# =========================================================
# BASE FUNNEL COMPLETA (1 fila por Id deuda) desde df_timeline_final
# Compatible con Google Colab y GitHub / ejecución local
# =========================================================

import pandas as pd
import numpy as np

# -----------------------------
# 0) Copia segura
# -----------------------------
df = df_timeline_final.copy()

# -----------------------------
# 1) Tipos / limpieza base
# -----------------------------
df["Id deuda"] = pd.to_numeric(df.get("Id deuda"), errors="coerce").astype("Int64")
df["Referencia"] = pd.to_numeric(df.get("Referencia"), errors="coerce").astype("Int64")

df["inserted_at"] = pd.to_datetime(df.get("inserted_at"), errors="coerce")

df["D_BRAVO"] = pd.to_numeric(df.get("D_BRAVO"), errors="coerce")
df["Ingreso_esperado"] = pd.to_numeric(df.get("Ingreso_esperado"), errors="coerce")
df["payment_to_bank"] = pd.to_numeric(df.get("payment_to_bank"), errors="coerce")

if "CATEGORIA_PRED" not in df.columns:
    raise ValueError("df_timeline_final no tiene la columna 'CATEGORIA_PRED'.")

df["_cat_all_norm"] = (
    df["CATEGORIA_PRED"]
    .astype("string")
    .str.strip()
    .str.upper()
)

# -----------------------------
# 2) Histórico liquidado
# -----------------------------
liq_hist = (
    df.groupby("Id deuda")["_cat_all_norm"]
      .apply(lambda s: (s == "LIQUIDADO").any())
)

# -----------------------------
# 3) Último registro por deuda
# -----------------------------
df = df.sort_values(["Id deuda", "inserted_at"], na_position="last")
df_ult = df.groupby("Id deuda", as_index=False).tail(1).copy()

df_ult = df_ult.rename(columns={
    "inserted_at": "inserted_at_ultima",
    "observations": "observations_ultima",
    "CATEGORIA_PRED": "CATEGORIA_PRED_ultima",
    "payment_to_bank": "payment_to_bank_ultima",
    "end": "end_ultima",
})

# -----------------------------
# 4) Flags observación / mes actual (Bogotá)
# -----------------------------
obs = (
    df_ult.get("observations_ultima", pd.Series(pd.NA, index=df_ult.index))
    .astype("string")
    .str.strip()
)
df_ult["tiene_obs"] = obs.notna() & (obs != "")

hoy = pd.Timestamp.now(tz="America/Bogota")
inicio_mes = hoy.replace(day=1, hour=0, minute=0, second=0, microsecond=0)

col = "inserted_at_ultima"
df_ult[col] = pd.to_datetime(df_ult[col], errors="coerce")

if df_ult[col].dt.tz is None:
    df_ult[col] = df_ult[col].dt.tz_localize(
        "America/Bogota",
        nonexistent="shift_forward",
        ambiguous="NaT"
    )
else:
    df_ult[col] = df_ult[col].dt.tz_convert("America/Bogota")

df_ult["es_este_mes"] = df_ult[col].notna() & (df_ult[col] >= inicio_mes)

# -----------------------------
# 5) Normalizar categoría última
# -----------------------------
df_ult["cat_norm"] = (
    df_ult["CATEGORIA_PRED_ultima"]
    .astype("string")
    .str.strip()
    .str.upper()
)

# -----------------------------
# 6) Liquidado definitivo
# -----------------------------
df_ult["tiene_liquidado_historico"] = df_ult["Id deuda"].map(liq_hist).fillna(False)
cond_f4 = df_ult["tiene_liquidado_historico"]

# -----------------------------
# 7) Fase 3 — Avance
# -----------------------------
es_avance_cat = df_ult["cat_norm"].isin(
    ["ACUERDO", "DESCUENTO", "CONTRAPROPUESTA"]
).fillna(False)

deuda = pd.to_numeric(df_ult["D_BRAVO"], errors="coerce")
pab = pd.to_numeric(df_ult["payment_to_bank_ultima"], errors="coerce")
cond_f3 = (es_avance_cat & ((deuda - pab) >= 10000)).fillna(False)

# -----------------------------
# 8) Fase 1 — Sin actualizar
# -----------------------------
cond_f1 = ((~df_ult["tiene_obs"]) | (~df_ult["es_este_mes"])).fillna(False)

# -----------------------------
# 9) Fase 2 — Actualizado
# -----------------------------
cond_f2 = ((~cond_f1) & (~cond_f3) & (~cond_f4)).fillna(False)

# -----------------------------
# 10) Asignar FASE
# -----------------------------
df_ult["FASE"] = np.select(
    [
        cond_f4.to_numpy(bool),
        cond_f3.to_numpy(bool),
        cond_f2.to_numpy(bool),
        cond_f1.to_numpy(bool),
    ],
    [
        "Fase 4 — Liquidado",
        "Fase 3 — Avance",
        "Fase 2 — Actualizado",
        "Fase 1 — Sin actualizar / antes de mes",
    ],
    default="Fase 2 — Actualizado"
)

# -----------------------------
# 11) STATUS
# -----------------------------
status_cat = df_ult["cat_norm"].str.title()

df_ult["STATUS"] = np.select(
    [
        cond_f4.to_numpy(bool),
        cond_f3.to_numpy(bool),
        cond_f2.to_numpy(bool),
        (~df_ult["tiene_obs"]).to_numpy(bool),
        (df_ult["tiene_obs"] & ~df_ult["es_este_mes"]).to_numpy(bool),
    ],
    [
        "Liquidado",
        status_cat,
        status_cat,
        "Sin actualización",
        "Actualizado antes",
    ],
    default=status_cat
)

# -----------------------------
# 12) Ingreso funnel
# -----------------------------
df_ult["Ingreso_funnel"] = np.where(
    cond_f4.to_numpy(bool),
    0,
    df_ult["Ingreso_esperado"].fillna(0)
)
# -----------------------------
# 13) Base final (con columnas requeridas)
# -----------------------------
cols_requeridas = [
    "Referencia",
    "Id deuda",
    "Cedula",
    "Nombre del cliente",
    "Negociador",
    "BANCOS_ESTANDAR",
    "Descuento",
    "D_BRAVO",
    "MORA"
    "Estructurable",
    "Potencial",
    "Meses en el Programa",
    "Tipo de Liquidacion",
    "Bucket",
    "Ahorro total",
    "Ahorro medio",
    "Por cobrar",
    "Potencial Credito",
    "Estado Deuda",
    "sub_estado_deuda",
    "estado_reparadora",
    "sub_estado_reparadora",
    "Mora_estructurado",
    "MORA_CREDITO",
    "Priority_level",
    "Ingreso_esperado",
    "Ultimo contacto",
    "fecha mensaje"
]

cols_eventos_ultima = [
    "inserted_at_ultima",
    "end_ultima",
    "CATEGORIA_PRED_ultima",
    "payment_to_bank_ultima",
    "observations_ultima",
]

cols_funnel = [
    "Ingreso_funnel",
    "FASE",
    "STATUS",
    "tiene_obs",
    "es_este_mes",
    "tiene_liquidado_historico",
]

# 1) Garantiza que existan (si no, NA)
for c in (cols_requeridas + cols_eventos_ultima + cols_funnel):
    if c not in df_ult.columns:
        df_ult[c] = pd.NA

# 2) Orden: primero requeridas, luego eventos, luego funnel, luego cualquier otra que exista
extras = [c for c in df_ult.columns if c not in (cols_requeridas + cols_eventos_ultima + cols_funnel)]
cols_salida = cols_requeridas + cols_eventos_ultima + cols_funnel + extras

df_base_funnel = df_ult[cols_salida].copy()

print("✅ df_base_funnel listo")
print("shape:", df_base_funnel.shape)
print("\nDistribución por FASE:")
print(df_base_funnel["FASE"].value_counts(dropna=False))

print("\nLiquidado (Fase 4) - filas:",
      (df_base_funnel["FASE"] == "Fase 4 — Liquidado").sum())

print(df_base_funnel.head(20))

✅ df_base_funnel listo
shape: (41279, 47)

Distribución por FASE:
FASE
Fase 1 — Sin actualizar / antes de mes    36776
Fase 2 — Actualizado                       3980
Fase 3 — Avance                             386
Fase 4 — Liquidado                          137
Name: count, dtype: int64

Liquidado (Fase 4) - filas: 137
    Referencia  Id deuda        Cedula                  Nombre del cliente  \
0      3012650    111266  5.221621e+07  CLAUDIA PATRICIA AVENDANO CALDERON   
1   3122962933    225561  5.086860e+07         Luz Nelly Betancourt Ortega   
2   3122962933    225562  5.086860e+07         Luz Nelly Betancourt Ortega   
3   3122962933    225563  5.086860e+07         Luz Nelly Betancourt Ortega   
4   3196892494    404243  5.290289e+07                Diana Marcela Suarez   
5   3183092531    427629  7.221792e+07          Faiber Alfonso Lemus Silva   
7   3003853673    434218  1.263591e+07                 Alfar de jesus Roca   
9   3003853673    434219  1.263591e+07                

In [25]:
df_timeline_final

,Referencia,Id deuda,Cedula,Nombre del cliente,Negociador,BANCOS_ESTANDAR,Descuento,D_BRAVO,Ingreso_esperado,MORA,...,fecha mensaje,inserted_at,end,payment_to_bank,CATEGORIA_PRED,observations,tipo_fila,Negociador liquidacion,Por?,Pago_banco_esperado
0,3012650,111266,5.221621e+07,CLAUDIA PATRICIA AVENDANO CALDERON,Negodito,Scotiabank Colpatria,0.7126,9256000.0,1.177355e+06,120.0,...,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.660174e+06
1,3122962933,225561,5.086860e+07,Luz Nelly Betancourt Ortega,William Santiago Abril Esguerra,Banco de Occidente,0.5000,2138165.0,1.908312e+05,55.0,...,2025-12-29 07:31:41,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.069082e+06
2,3122962933,225562,5.086860e+07,Luz Nelly Betancourt Ortega,William Santiago Abril Esguerra,Banco de Occidente,0.5600,3299116.0,3.297796e+05,55.0,...,2025-12-29 07:31:41,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.451611e+06
3,3122962933,225563,5.086860e+07,Luz Nelly Betancourt Ortega,William Santiago Abril Esguerra,Tuya,0.6715,5959400.0,7.143101e+05,55.0,...,2025-12-29 07:31:41,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.957663e+06
4,3196892494,404243,5.290289e+07,Diana Marcela Suarez,Dayana Isabel Ojito Ortiz,Banco Popular,0.4800,1921650.0,1.646470e+05,79.0,...,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9.992580e+05
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
43173,3225833196,2091518,1.019066e+09,JUAN DAVID LOZANO TRIVINO,Gabriela Saavedra Latorre,Banco Davivienda,0.3637,13695024.0,8.890871e+05,7.0,...,2026-01-09 09:44:37,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8.714144e+06
43174,3225833196,2091519,1.019066e+09,JUAN DAVID LOZANO TRIVINO,Gabriela Saavedra Latorre,Scotiabank Colpatria,0.5000,50463309.0,4.503850e+06,7.0,...,2026-01-09 09:44:37,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.523165e+07
43175,3225833196,2091520,1.019066e+09,JUAN DAVID LOZANO TRIVINO,Gabriela Saavedra Latorre,Bancolombia,0.4700,82022139.0,6.881247e+06,7.0,...,2026-01-09 09:44:37,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.347173e+07
43176,3105507630,2091523,7.938104e+07,JUAN CARLOS PACHECO CONTRERAS,Julian Andres Ossa Bohorquez,Bancolombia,0.4700,130935011.0,1.098479e+07,7.0,...,2026-01-09 10:51:57,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6.939556e+07


In [26]:
df_base_funnel.info()

<class 'pandas.core.frame.DataFrame'>
Index: 41279 entries, 0 to 43177
Data columns (total 47 columns):
 #   Column                     Non-Null Count  Dtype                         
---  ------                     --------------  -----                         
 0   Referencia                 41279 non-null  Int64                         
 1   Id deuda                   41279 non-null  Int64                         
 2   Cedula                     41215 non-null  float64                       
 3   Nombre del cliente         41215 non-null  object                        
 4   Negociador                 41266 non-null  object                        
 5   BANCOS_ESTANDAR            41279 non-null  object                        
 6   Descuento                  36308 non-null  float64                       
 7   D_BRAVO                    41279 non-null  float64                       
 8   MORAEstructurable          0 non-null      object                        
 9   Potencial             

In [27]:
df_timeline_final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 43178 entries, 0 to 43177
Data columns (total 37 columns):
 #   Column                  Non-Null Count  Dtype                         
---  ------                  --------------  -----                         
 0   Referencia              43178 non-null  Int64                         
 1   Id deuda                43178 non-null  Int64                         
 2   Cedula                  43041 non-null  float64                       
 3   Nombre del cliente      43041 non-null  object                        
 4   Negociador              43157 non-null  object                        
 5   BANCOS_ESTANDAR         43178 non-null  object                        
 6   Descuento               38197 non-null  float64                       
 7   D_BRAVO                 43178 non-null  float64                       
 8   Ingreso_esperado        38197 non-null  float64                       
 9   MORA                    43041 non-null  float64   

In [28]:
# =====================================================
# Google Sheets en Colab + GitHub usando MI_JSON
# - En Colab: lee MI_JSON desde Secrets (userdata)
# - En GitHub / local: lee MI_JSON desde variable de entorno o archivo .json
# =====================================================

!pip install -q gspread gspread_dataframe

import os, json
import pandas as pd
import gspread
from gspread_dataframe import get_as_dataframe

def _load_service_account_info():
    """
    Devuelve el dict del service account.
    Prioridad:
      1) Colab: userdata.get("MI_JSON")  (tu regla)
      2) Env var: MI_JSON (json string) o MI_JSON_PATH (ruta a .json)
      3) Archivo: service_account.json en el repo (si existe)
    """
    # 1) Colab (seguro si existe)
    try:
        from google.colab import userdata  # solo existe en Colab
        mi_json = userdata.get("MI_JSON")  # <-- tal como lo pediste
        if mi_json:
            return json.loads(mi_json) if isinstance(mi_json, str) else mi_json
    except Exception:
        pass

    # 2) GitHub/Local: env var MI_JSON (contenido json)
    mi_json_env = os.getenv("MI_JSON")
    if mi_json_env:
        return json.loads(mi_json_env)

    # 2b) GitHub/Local: env var MI_JSON_PATH (ruta a archivo json)
    mi_json_path = os.getenv("MI_JSON_PATH")
    if mi_json_path and os.path.exists(mi_json_path):
        with open(mi_json_path, "r", encoding="utf-8") as f:
            return json.load(f)

    # 3) Fallback: archivo en el repo (NO recomendado si es privado, pero útil local)
    default_path = "service_account.json"
    if os.path.exists(default_path):
        with open(default_path, "r", encoding="utf-8") as f:
            return json.load(f)

    raise RuntimeError(
        "No encontré credenciales. En Colab asegúrate de tener el Secret MI_JSON. "
        "En GitHub configura env var MI_JSON (json) o MI_JSON_PATH (ruta)."
    )

def get_gspread_client():
    sa_info = _load_service_account_info()
    return gspread.service_account_from_dict(sa_info)

# =====================================================
# Leer la hoja "Asignacion"
# =====================================================
SPREADSHEET_ID = "184zZcHRajscoJRyjMkkQjPh0qOlY3iVkQICKHl-iOyU"
NOMBRE_HOJA = "Asignacion"

gc = get_gspread_client()
sh = gc.open_by_key(SPREADSHEET_ID)
ws = sh.worksheet(NOMBRE_HOJA)

df_asignacion = get_as_dataframe(ws, evaluate_formulas=True).dropna(how="all")
df_asignacion.columns = df_asignacion.columns.str.strip()

df_asignacion

,Negociador,Tipo
0,Julian Andres Ossa Bohorquez,Back
1,Gabriela Saavedra Latorre,Back
2,Yithza Camila Paez Lopez,Estructurados
3,Alba Yohana Moreno Martin,Premium
4,Maria Daniela Sarta Sanchez,Premium
5,Francy Tatiana Sanchez Fontecha,Premium
6,Katherine Marulanda Bernal,Premium
7,William Santiago Abril Esguerra,Premium
8,Niyiret Julio Santos,Premium
9,Angie Lizeth Cubides Neira,Premium


In [29]:
df_timeline_final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 43178 entries, 0 to 43177
Data columns (total 37 columns):
 #   Column                  Non-Null Count  Dtype                         
---  ------                  --------------  -----                         
 0   Referencia              43178 non-null  Int64                         
 1   Id deuda                43178 non-null  Int64                         
 2   Cedula                  43041 non-null  float64                       
 3   Nombre del cliente      43041 non-null  object                        
 4   Negociador              43157 non-null  object                        
 5   BANCOS_ESTANDAR         43178 non-null  object                        
 6   Descuento               38197 non-null  float64                       
 7   D_BRAVO                 43178 non-null  float64                       
 8   Ingreso_esperado        38197 non-null  float64                       
 9   MORA                    43041 non-null  float64   

Aquí empezamos con los Bucket

In [30]:
import numpy as np
import pandas as pd

# =========================================================
# 1) Descuento_Actualizacion
#    Si payment_to_bank no es nulo: ((payment_to_bank/D_BRAVO)-1)*-1
# =========================================================
df_timeline_final["Descuento_Actualizacion"] = np.where(
    df_timeline_final["payment_to_bank"].notna(),
    ((df_timeline_final["payment_to_bank"] / df_timeline_final["D_BRAVO"]) - 1) * -1,
    np.nan
)

# =========================================================
# 2) Tipo de Actividad (reglas)
# =========================================================
cat = df_timeline_final["CATEGORIA_PRED"]
desc = df_timeline_final["Descuento_Actualizacion"]

cats_efectiva = {"ACUERDO", "DESCUENTO", "EN_CREDITO", "Liquidado", "SALDOS", "DEPOSITO"}
rango_desc = desc.between(0.03, 0.98, inclusive="both")

# EFECTIVA
cond_efectiva = cat.isin(cats_efectiva) & rango_desc

# ACTUALIZADO_ALIANZAS
cond_act_alianzas = (cat == "ACTUALIZADO_ALIANZAS")

# EFECTIVA DESCUENTO ALIANZAS:
# payment_to_bank < D_BRAVO y (D_BRAVO - payment_to_bank) <= 5000
diff = df_timeline_final["D_BRAVO"] - df_timeline_final["payment_to_bank"]
cond_desc_alianzas = (
    cond_act_alianzas &
    (df_timeline_final["payment_to_bank"] < df_timeline_final["D_BRAVO"]) &
    (diff <= 5000)
)

df_timeline_final["Tipo de Actividad"] = np.select(
    [cond_efectiva, cond_desc_alianzas, cond_act_alianzas],
    ["EFECTIVA", "EFECTIVA DESCUENTO ALIANZAS", "EFECTIVA ALIANZAS"],
    default="NO EFECTIVA"
)

# =========================================================
# 3) Si payment_to_bank, CATEGORIA_PRED y observations son NaN -> Tipo de Actividad = NaN
# =========================================================
cond_nan_total = (
    df_timeline_final["payment_to_bank"].isna() &
    df_timeline_final["CATEGORIA_PRED"].isna() &
    df_timeline_final["observations"].isna()
)
df_timeline_final.loc[cond_nan_total, "Tipo de Actividad"] = np.nan

# =========================================================
# 4) Insertar ambas columnas al lado de 'observations'
#    (quedan inmediatamente después de observations, en este orden)
# =========================================================
idx_obs = df_timeline_final.columns.get_loc("observations")

# Pop + insert para asegurar ubicación/orden
col_da = df_timeline_final.pop("Descuento_Actualizacion")
col_ta = df_timeline_final.pop("Tipo de Actividad")

df_timeline_final.insert(idx_obs + 1, "Descuento_Actualizacion", col_da)
df_timeline_final.insert(idx_obs + 2, "Tipo de Actividad", col_ta)

In [31]:
df_base_funnel.columns

Index(['Referencia', 'Id deuda', 'Cedula', 'Nombre del cliente', 'Negociador',
       'BANCOS_ESTANDAR', 'Descuento', 'D_BRAVO', 'MORAEstructurable',
       'Potencial', 'Meses en el Programa', 'Tipo de Liquidacion', 'Bucket',
       'Ahorro total', 'Ahorro medio', 'Por cobrar', 'Potencial Credito',
       'Estado Deuda', 'sub_estado_deuda', 'estado_reparadora',
       'sub_estado_reparadora', 'Mora_estructurado', 'MORA_CREDITO',
       'Priority_level', 'Ingreso_esperado', 'Ultimo contacto',
       'fecha mensaje', 'inserted_at_ultima', 'end_ultima',
       'CATEGORIA_PRED_ultima', 'payment_to_bank_ultima',
       'observations_ultima', 'Ingreso_funnel', 'FASE', 'STATUS', 'tiene_obs',
       'es_este_mes', 'tiene_liquidado_historico', 'MORA', 'Estructurable',
       'ultimo contacto', 'tipo_fila', 'Negociador liquidacion', 'Por?',
       'Pago_banco_esperado', '_cat_all_norm', 'cat_norm'],
      dtype='object')

In [32]:
import numpy as np
import pandas as pd

# =========================================================
# 1) Descuento_Actualizacion
#    Si payment_to_bank no es nulo: ((payment_to_bank/D_BRAVO)-1)*-1
# =========================================================
df_base_funnel["Descuento_Actualizacion"] = np.where(
    df_base_funnel["payment_to_bank_ultima"].notna(),
    ((df_base_funnel["payment_to_bank_ultima"] / df_base_funnel["D_BRAVO"]) - 1) * -1,
    np.nan
)

# =========================================================
# 2) Tipo de Actividad (reglas)
# =========================================================
cat = df_base_funnel["CATEGORIA_PRED_ultima"]
desc = df_base_funnel["Descuento_Actualizacion"]

cats_efectiva = {"ACUERDO", "DESCUENTO", "EN_CREDITO", "Liquidado", "SALDOS", "DEPOSITO"}
rango_desc = desc.between(0.03, 0.98, inclusive="both")

# EFECTIVA
cond_efectiva = cat.isin(cats_efectiva) & rango_desc

# ACTUALIZADO_ALIANZAS
cond_act_alianzas = (cat == "ACTUALIZADO_ALIANZAS")

# EFECTIVA DESCUENTO ALIANZAS:
# payment_to_bank < D_BRAVO y (D_BRAVO - payment_to_bank) <= 5000
diff = df_base_funnel["D_BRAVO"] - df_base_funnel["payment_to_bank_ultima"]
cond_desc_alianzas = (
    cond_act_alianzas &
    (df_base_funnel["payment_to_bank_ultima"] < df_base_funnel["D_BRAVO"]) &
    (diff <= 5000)
)

df_base_funnel["Tipo de Actividad"] = np.select(
    [cond_efectiva, cond_desc_alianzas, cond_act_alianzas],
    ["EFECTIVA", "EFECTIVA DESCUENTO ALIANZAS", "EFECTIVA ALIANZAS"],
    default="NO EFECTIVA"
)

# =========================================================
# 3) Si payment_to_bank, CATEGORIA_PRED y observations son NaN -> Tipo de Actividad = NaN
# =========================================================
cond_nan_total = (
    df_base_funnel["payment_to_bank_ultima"].isna() &
    df_base_funnel["CATEGORIA_PRED_ultima"].isna() &
    df_base_funnel["observations_ultima"].isna()
)
df_base_funnel.loc[cond_nan_total, "Tipo de Actividad"] = np.nan

# =========================================================
# 4) Insertar ambas columnas al lado de 'observations'
#    (quedan inmediatamente después de observations, en este orden)
# =========================================================
idx_obs = df_base_funnel.columns.get_loc("observations_ultima")

# Pop + insert para asegurar ubicación/orden
col_da = df_base_funnel.pop("Descuento_Actualizacion")
col_ta = df_base_funnel.pop("Tipo de Actividad")

df_base_funnel.insert(idx_obs + 1, "Descuento_Actualizacion", col_da)
df_base_funnel.insert(idx_obs + 2, "Tipo de Actividad", col_ta)

In [33]:
df_base_funnel = df_base_funnel.drop(columns=["Bucket"], errors="ignore")

In [34]:
df_timeline_final = df_timeline_final.drop(columns=["Bucket"], errors="ignore")

In [35]:
df_timeline_final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 43178 entries, 0 to 43177
Data columns (total 38 columns):
 #   Column                   Non-Null Count  Dtype                         
---  ------                   --------------  -----                         
 0   Referencia               43178 non-null  Int64                         
 1   Id deuda                 43178 non-null  Int64                         
 2   Cedula                   43041 non-null  float64                       
 3   Nombre del cliente       43041 non-null  object                        
 4   Negociador               43157 non-null  object                        
 5   BANCOS_ESTANDAR          43178 non-null  object                        
 6   Descuento                38197 non-null  float64                       
 7   D_BRAVO                  43178 non-null  float64                       
 8   Ingreso_esperado         38197 non-null  float64                       
 9   MORA                     43041 non-null

In [36]:
df_timeline_final

,Referencia,Id deuda,Cedula,Nombre del cliente,Negociador,BANCOS_ESTANDAR,Descuento,D_BRAVO,Ingreso_esperado,MORA,...,end,payment_to_bank,CATEGORIA_PRED,observations,Descuento_Actualizacion,Tipo de Actividad,tipo_fila,Negociador liquidacion,Por?,Pago_banco_esperado
0,3012650,111266,5.221621e+07,CLAUDIA PATRICIA AVENDANO CALDERON,Negodito,Scotiabank Colpatria,0.7126,9256000.0,1.177355e+06,120.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.660174e+06
1,3122962933,225561,5.086860e+07,Luz Nelly Betancourt Ortega,William Santiago Abril Esguerra,Banco de Occidente,0.5000,2138165.0,1.908312e+05,55.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.069082e+06
2,3122962933,225562,5.086860e+07,Luz Nelly Betancourt Ortega,William Santiago Abril Esguerra,Banco de Occidente,0.5600,3299116.0,3.297796e+05,55.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.451611e+06
3,3122962933,225563,5.086860e+07,Luz Nelly Betancourt Ortega,William Santiago Abril Esguerra,Tuya,0.6715,5959400.0,7.143101e+05,55.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.957663e+06
4,3196892494,404243,5.290289e+07,Diana Marcela Suarez,Dayana Isabel Ojito Ortiz,Banco Popular,0.4800,1921650.0,1.646470e+05,79.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9.992580e+05
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
43173,3225833196,2091518,1.019066e+09,JUAN DAVID LOZANO TRIVINO,Gabriela Saavedra Latorre,Banco Davivienda,0.3637,13695024.0,8.890871e+05,7.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8.714144e+06
43174,3225833196,2091519,1.019066e+09,JUAN DAVID LOZANO TRIVINO,Gabriela Saavedra Latorre,Scotiabank Colpatria,0.5000,50463309.0,4.503850e+06,7.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.523165e+07
43175,3225833196,2091520,1.019066e+09,JUAN DAVID LOZANO TRIVINO,Gabriela Saavedra Latorre,Bancolombia,0.4700,82022139.0,6.881247e+06,7.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.347173e+07
43176,3105507630,2091523,7.938104e+07,JUAN CARLOS PACHECO CONTRERAS,Julian Andres Ossa Bohorquez,Bancolombia,0.4700,130935011.0,1.098479e+07,7.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6.939556e+07


In [37]:
import numpy as np
import pandas as pd

# =========================================================
# CONFIG
# =========================================================
COL_ULT_CONTACTO = "ultimo contacto"   # <- así, con espacio
COL_NEGOCIADOR = "Negociador"          # <- AJUSTA si tu columna se llama distinto
COL_BANCO = "BANCOS_ESTANDAR"

# Distribución objetivo por negociador
TARGET_PCTS = {0: 0.10, 1: 0.10, 2: 0.15, 3: 0.15, 4: 0.25, 5: 0.25}
BUCKETS = [0, 1, 2, 3, 4, 5]

# Pesos para score final (puedes ajustarlos)
W_BASE = 0.75   # peso de lo ya construido (Bucket_row consolidado)
W_BANK = 0.25   # peso del banco
# Ahorro/D_BRAVO entra como impulso suave por buckets (ver más abajo)

# Más peso al mejor banco que al peor (para el criterio banco)
ALPHA_BEST_BANK = 0.70

# =========================================================
# 0) COPIA + LIMPIEZA / NORMALIZACIÓN
# =========================================================
df_timeline_final = df_timeline_final.copy()

# Numéricas
df_timeline_final[COL_ULT_CONTACTO] = pd.to_numeric(df_timeline_final[COL_ULT_CONTACTO], errors="coerce").fillna(6)
df_timeline_final["Potencial"] = pd.to_numeric(df_timeline_final["Potencial"], errors="coerce")
df_timeline_final["Estructurable"] = pd.to_numeric(df_timeline_final["Estructurable"], errors="coerce")
df_timeline_final["MORA"] = pd.to_numeric(df_timeline_final["MORA"], errors="coerce")
df_timeline_final["Meses en el Programa"] = pd.to_numeric(df_timeline_final["Meses en el Programa"], errors="coerce")
df_timeline_final["D_BRAVO"] = pd.to_numeric(df_timeline_final["D_BRAVO"], errors="coerce")
df_timeline_final["Ahorro total"] = pd.to_numeric(df_timeline_final["Ahorro total"], errors="coerce")

# Textos
mora_credito = df_timeline_final["MORA_CREDITO"].astype("string").str.strip().str.lower()
mora_estruct = df_timeline_final["Mora_estructurado"].astype("string").str.strip().str.lower()
cat_pred = df_timeline_final["CATEGORIA_PRED"].astype("string").str.strip()
cat_upper = cat_pred.str.upper()

# Potencial Credito mixto
pc_raw = df_timeline_final["Potencial Credito"].astype("string").str.strip()
pc_lower = pc_raw.str.lower()
pc_num = pd.to_numeric(pc_raw, errors="coerce")

# =========================================================
# 1) BUCKET "POR FILA" (PASO INTERMEDIO) => Bucket_row (0..5)
# =========================================================
# 1.1 Base por recencia (dinámico 0..5)
s = df_timeline_final[COL_ULT_CONTACTO]
base = pd.qcut(s.rank(method="first"), q=6, labels=False, duplicates="drop")
n_bins = int(base.max() + 1) if base.notna().any() else 1
if n_bins > 1:
    base_scaled = np.floor(base * (5 / (n_bins - 1))).astype(int)
else:
    base_scaled = pd.Series(0, index=df_timeline_final.index, dtype=int)
df_timeline_final["Bucket_row"] = base_scaled.astype("Int64")

# 1.2 Penalización por cliente con al menos 1 "Liquidado" (suave +1)
clientes_con_liquidado = df_timeline_final.loc[cat_pred.eq("Liquidado"), "Referencia"].dropna().unique()
mask_cliente_liquidado = df_timeline_final["Referencia"].isin(clientes_con_liquidado)
df_timeline_final.loc[mask_cliente_liquidado, "Bucket_row"] = (
    df_timeline_final.loc[mask_cliente_liquidado, "Bucket_row"].astype(int) + 1
).clip(upper=5).astype("Int64")

# 1.3 Mora_estructurado
cond_estruct_si = mora_estruct.isin(["sí", "si"])
cond_estruct_no = mora_estruct.eq("no")

df_timeline_final.loc[cond_estruct_si, "Bucket_row"] = (
    df_timeline_final.loc[cond_estruct_si, "Bucket_row"].astype(int).clip(lower=4)
).astype("Int64")

df_timeline_final.loc[cond_estruct_no & ~cond_estruct_si, "Bucket_row"] = (
    df_timeline_final.loc[cond_estruct_no & ~cond_estruct_si, "Bucket_row"].astype(int).clip(lower=3)
).astype("Int64")

# 1.4 Potencial (alto aleja => peor (máximo))
p = df_timeline_final["Potencial"]
mask_p = p.notna()
if mask_p.any():
    p_bins = pd.qcut(p[mask_p].rank(method="first"), q=6, labels=False, duplicates="drop")
    n_pbins = int(p_bins.max() + 1) if p_bins.notna().any() else 1
    if n_pbins > 1:
        p_scaled = np.floor(p_bins * (5 / (n_pbins - 1))).astype(int)
    else:
        p_scaled = pd.Series(0, index=p_bins.index, dtype=int)

    bucket_por_pot = pd.Series(pd.NA, index=df_timeline_final.index, dtype="Int64")
    bucket_por_pot.loc[p_scaled.index] = p_scaled.astype("Int64")

    df_timeline_final.loc[mask_p, "Bucket_row"] = np.maximum(
        df_timeline_final.loc[mask_p, "Bucket_row"].astype(int),
        bucket_por_pot.loc[mask_p].astype(int),
    ).astype("Int64")

# 1.5 Estructurable (impulso suave)
mask_e = df_timeline_final["Estructurable"].isin([0, 1])
mask_e1 = mask_e & (df_timeline_final["Estructurable"] == 1)
mask_e0 = mask_e & (df_timeline_final["Estructurable"] == 0)

df_timeline_final.loc[mask_e1, "Bucket_row"] = (
    df_timeline_final.loc[mask_e1, "Bucket_row"].astype(int) - 1
).clip(lower=0).astype("Int64")

df_timeline_final.loc[mask_e0, "Bucket_row"] = (
    df_timeline_final.loc[mask_e0, "Bucket_row"].astype(int) + 1
).clip(upper=5).astype("Int64")

# 1.6 Potencial Credito (mixto)
mask_pc_num = pc_num.notna()
if mask_pc_num.any():
    pc_bins = pd.qcut(pc_num[mask_pc_num].rank(method="first"), q=6, labels=False, duplicates="drop")
    n_pcbins = int(pc_bins.max() + 1) if pc_bins.notna().any() else 1
    if n_pcbins > 1:
        pc_scaled = np.floor(pc_bins * (5 / (n_pcbins - 1))).astype(int)
    else:
        pc_scaled = pd.Series(0, index=pc_bins.index, dtype=int)

    pc_score = pd.Series(pd.NA, index=df_timeline_final.index, dtype="Int64")
    pc_score.loc[pc_scaled.index] = pc_scaled.astype("Int64")

    # score 0-1 => mejora (-1)
    mask_pc_bajo = mask_pc_num & pc_score.isin([0, 1])
    df_timeline_final.loc[mask_pc_bajo, "Bucket_row"] = (
        df_timeline_final.loc[mask_pc_bajo, "Bucket_row"].astype(int) - 1
    ).clip(lower=0).astype("Int64")

    # score 4-5 => empeora (+1)
    mask_pc_alto = mask_pc_num & pc_score.isin([4, 5])
    df_timeline_final.loc[mask_pc_alto, "Bucket_row"] = (
        df_timeline_final.loc[mask_pc_alto, "Bucket_row"].astype(int) + 1
    ).clip(upper=5).astype("Int64")

# Tradicional o NaN => intermedio (mínimo 3)
mask_pc_trad = pc_lower.eq("tradicional")
mask_pc_nan = pc_raw.isna() | pc_lower.isin(["<na>", "nan", "none", ""])
mask_pc_intermedio = (mask_pc_trad | mask_pc_nan) & (~mask_pc_num)
df_timeline_final.loc[mask_pc_intermedio, "Bucket_row"] = (
    df_timeline_final.loc[mask_pc_intermedio, "Bucket_row"].astype(int).clip(lower=3)
).astype("Int64")

# 1.7 Meses en el Programa + MORA
mora = df_timeline_final["MORA"]
meses_prog = df_timeline_final["Meses en el Programa"]

# (A) MORA < 5 => grave => 4 o 5 dinámico
mask_mora_baja = mora.notna() & (mora < 5)
if mask_mora_baja.any():
    mora_baja = mora[mask_mora_baja]
    mb = pd.qcut(mora_baja.rank(method="first"), q=2, labels=False, duplicates="drop")
    if mb.notna().any() and int(mb.max() + 1) > 1:
        mb_scaled = mb.map({0: 4, 1: 5}).astype(int)
    else:
        mb_scaled = pd.Series(4, index=mora_baja.index, dtype=int)

    df_timeline_final.loc[mb_scaled.index, "Bucket_row"] = np.maximum(
        df_timeline_final.loc[mb_scaled.index, "Bucket_row"].astype(int),
        mb_scaled.astype(int),
    ).astype("Int64")

# (B) MORA > 15 => intermedio => 3 o 4 dinámico
mask_mora_alta = mora.notna() & (mora > 15)
if mask_mora_alta.any():
    mora_alta = mora[mask_mora_alta]
    ma = pd.qcut(mora_alta.rank(method="first"), q=2, labels=False, duplicates="drop")
    if ma.notna().any() and int(ma.max() + 1) > 1:
        ma_scaled = ma.map({0: 3, 1: 4}).astype(int)
    else:
        ma_scaled = pd.Series(3, index=mora_alta.index, dtype=int)

    df_timeline_final.loc[ma_scaled.index, "Bucket_row"] = np.maximum(
        df_timeline_final.loc[ma_scaled.index, "Bucket_row"].astype(int),
        ma_scaled.astype(int),
    ).astype("Int64")

# (C) MORA 6..15 y Meses>3 => impulso positivo -1
mask_top = mora.notna() & meses_prog.notna() & mora.between(6, 15, inclusive="both") & (meses_prog > 3)
df_timeline_final.loc[mask_top, "Bucket_row"] = (
    df_timeline_final.loc[mask_top, "Bucket_row"].astype(int) - 1
).clip(lower=0).astype("Int64")

# =========================================================
# 2) REGLAS DURAS (BUCKET 5 FIJO) — POR REFERENCIA
# =========================================================
cond_credito_si = mora_credito.isin(["sí", "si"])
refs_credito_si = df_timeline_final.loc[cond_credito_si, "Referencia"].dropna().unique()

refs_no_viable_demanda = df_timeline_final.loc[
    cat_upper.isin(["NO_VIABLE", "DEMANDA", "ILOCALIZADO", "PAGO_POR_FUERA"]),
    "Referencia"
].dropna().unique()

refs_hard_5 = set(refs_credito_si).union(set(refs_no_viable_demanda))

# =========================================================
# 3) BANCOS_ESTANDAR (POR REFERENCIA) — PEGA TU LISTA BUENO->MALO
# =========================================================
bancos_orden = [
    "Cobrando",
    "Rappipay",
    "Alkomprar",
    "Rappicard",
    "Pichincha",
    "Bancolombia",
    "Scotiabank Colpatria",
    "Rapicredit",
    "Bancoomeva",
    "Codensa",
    "Covinoc",
    "Zinobe",
    "Banco Popular",
    "SisteCredito",
    "Banco AV Villas",
    "Colsubsidio",
    "Refinancia",
    "Tuya",
    "Credijamar",
    "Agaval",
    "Alkosto",
    "Banco Falabella",
    "Banco de Bogotá",
    "Sistemcobro",
    "Bancamia",
    "Banco Finandina",
    "Itaú",
    "FGA",
    "BBVA Colombia",
    "Serlefin",
    "Flamingo",
    "Mundo Mujer",
    "Banco Davivienda",
    "Banco Caja Social",
    "Comultrasan",
    "Juriscoop",
    "Credivalores",
    "Banco de Occidente",
    "Compensar",
    "JOHN",
    "Fincomercio",
    "GNB Sudameris",
    "Serfinanza",
    "Nu",
    "LuloBank",
    "Aslegal",
    "Baninca",
    "Coltefinanciera",
    "Confiar",
    "Contacto solución",
    "Garantias Comunitarias",
    "GRUPO JURIDICO DEUDU",
    "Inversionistas Estratégicos",
    "Juancho te Presta",
    "QNT SAS",
]

bancos_norm = df_timeline_final[COL_BANCO].astype("string").str.strip().str.lower()

if len(bancos_orden) > 0:
    banco_rank_map = {b.strip().lower(): i for i, b in enumerate(bancos_orden)}
    rank_intermedio = len(bancos_orden) // 2
    df_timeline_final["_banco_rank"] = bancos_norm.map(banco_rank_map).fillna(rank_intermedio).astype(int)
else:
    df_timeline_final["_banco_rank"] = 0

grp_rank = df_timeline_final.groupby("Referencia")["_banco_rank"]
best_rank = grp_rank.min()
worst_rank = grp_rank.max()

score_banco = (ALPHA_BEST_BANK * best_rank) + ((1 - ALPHA_BEST_BANK) * worst_rank)

tmp = score_banco.rank(method="first")
bb = pd.qcut(tmp, q=6, labels=False, duplicates="drop")
n_bb = int(bb.max() + 1) if bb.notna().any() else 1
if n_bb > 1:
    bucket_banco = np.floor(bb * (5 / (n_bb - 1))).astype(int)
else:
    bucket_banco = pd.Series(0, index=score_banco.index, dtype=int)
bucket_banco = bucket_banco.clip(0, 5).astype("Int64")
bucket_banco.name = "Bucket_Banco"

# =========================================================
# 4) CRITERIO: max(Ahorro total / D_BRAVO) POR REFERENCIA
#    (alto ratio => empuja hacia bucket bajo, bajo ratio => hacia bucket alto)
# =========================================================
df_timeline_final["ratio_ahorro_db"] = np.where(
    (df_timeline_final["D_BRAVO"].notna()) & (df_timeline_final["D_BRAVO"] > 0) & df_timeline_final["Ahorro total"].notna(),
    df_timeline_final["Ahorro total"] / df_timeline_final["D_BRAVO"],
    np.nan
)

ratio_max_ref = df_timeline_final.groupby("Referencia")["ratio_ahorro_db"].max()

mask_ratio = ratio_max_ref.notna()
if mask_ratio.any():
    rr = pd.qcut(ratio_max_ref[mask_ratio].rank(method="first"), q=6, labels=False, duplicates="drop")
    n_rr = int(rr.max() + 1) if rr.notna().any() else 1
    if n_rr > 1:
        rr_scaled = np.floor(rr * (5 / (n_rr - 1))).astype(int)
    else:
        rr_scaled = pd.Series(0, index=rr.index, dtype=int)

    bucket_ratio = pd.Series(pd.NA, index=ratio_max_ref.index, dtype="Int64")
    bucket_ratio.loc[rr_scaled.index] = (5 - rr_scaled).astype("Int64")  # invertido: alto ratio => bucket bajo
else:
    bucket_ratio = pd.Series(pd.NA, index=ratio_max_ref.index, dtype="Int64")

bucket_ratio.name = "Bucket_Ratio"

# =========================================================
# 5) TABLA POR REFERENCIA: SCORE CONTINUO (para ordenar dentro de negociador)
# =========================================================
bucket_ref_base = df_timeline_final.groupby("Referencia")["Bucket_row"].max().astype("Int64")
bucket_ref_base.name = "bucket_ref_base"

ref_table = pd.concat([bucket_ref_base, bucket_banco, bucket_ratio], axis=1)
ref_table.columns = ["bucket_ref_base", "Bucket_Banco", "Bucket_Ratio"]

# Fill intermedios si falta info (NO castiga ni premia)
ref_table["bucket_ref_base"] = ref_table["bucket_ref_base"].fillna(0)
ref_table["Bucket_Banco"] = ref_table["Bucket_Banco"].fillna(3)
ref_table["Bucket_Ratio"] = ref_table["Bucket_Ratio"].fillna(3)

# score continuo: menor = más importante
# (ratio ya viene como bucket 0..5 invertido, así que entra directo)
ref_table["score_ref"] = (
    (W_BASE * ref_table["bucket_ref_base"].astype(float)) +
    (W_BANK * ref_table["Bucket_Banco"].astype(float)) +
    (0.20 * ref_table["Bucket_Ratio"].astype(float))  # impulso ahorro/DBRAVO
)
# =========================================================
# 5.1) REGRA DURA: D_BRAVO MUY ALTO => BUCKET 0 o 1
# =========================================================

# promedio global de D_BRAVO
mean_dbravo = df_timeline_final["D_BRAVO"].mean()

# referencias con al menos una deuda 90% > promedio
refs_dbravo_alto = (
    df_timeline_final.loc[
        df_timeline_final["D_BRAVO"] >= 1.9 * mean_dbravo,
        "Referencia"
    ]
    .dropna()
    .unique()
)

# referencias con Banco Davivienda
refs_davivienda = (
    df_timeline_final.loc[
        df_timeline_final[COL_BANCO]
        .astype("string")
        .str.strip()
        .str.lower()
        .eq("banco davivienda"),
        "Referencia"
    ]
    .dropna()
    .unique()
)

# elegibles: alto D_BRAVO, no hard 5, no Davivienda
refs_dbravo_prioridad = (
    set(refs_dbravo_alto)
    - set(refs_hard_5)
    - set(refs_davivienda)
)

# aplicar regla:
# si bucket_base <= 2 => Bucket 0
# si bucket_base > 2  => Bucket 1
mask_prior = ref_table.index.isin(refs_dbravo_prioridad)

ref_table.loc[mask_prior & (ref_table["bucket_ref_base"] <= 2), "bucket_ref_base"] = 0
ref_table.loc[mask_prior & (ref_table["bucket_ref_base"] > 2), "bucket_ref_base"] = 1


# =========================================================
# 6) MAPEAR NEGOCIADOR POR REFERENCIA
#    (si una referencia aparece con varios negociadores, toma el más frecuente)
# =========================================================
ref_neg = (
    df_timeline_final.groupby(["Referencia", COL_NEGOCIADOR]).size()
    .reset_index(name="n")
    .sort_values(["Referencia", "n"], ascending=[True, False])
    .drop_duplicates("Referencia")
    .set_index("Referencia")[COL_NEGOCIADOR]
)

ref_table = ref_table.join(ref_neg.rename("negociador_ref"), how="left")

# =========================================================
# 7) ASIGNACIÓN EQUITATIVA POR NEGOCIADOR (CUOTAS)
#    Respeta hard=5. El resto se distribuye por score (mejor score => bucket más bajo).
# =========================================================
def _target_counts(n_total: int) -> dict:
    # floor + repartir sobrantes por los buckets más importantes primero
    raw = {b: int(np.floor(TARGET_PCTS[b] * n_total)) for b in BUCKETS}
    missing = n_total - sum(raw.values())
    # repartir sobrantes: 0,1,2,3,4,5
    for b in BUCKETS:
        if missing <= 0:
            break
        raw[b] += 1
        missing -= 1
    return raw

bucket_final = pd.Series(index=ref_table.index, dtype="Int64")

for neg, sub in ref_table.groupby("negociador_ref", dropna=False):
    refs = sub.index.tolist()
    n_total = len(refs)
    if n_total == 0:
        continue

    targets = _target_counts(n_total)

    # hard 5 dentro del negociador
    hard_mask = [r in refs_hard_5 for r in refs]
    hard_refs = sub.index[hard_mask]
    n_hard = len(hard_refs)

    # asignar hard primero
    if n_hard > 0:
        bucket_final.loc[hard_refs] = 5

    # refs elegibles (no hard)
    free = sub.loc[~pd.Index(refs).isin(hard_refs)].copy()
    n_free = len(free)

    if n_free == 0:
        continue

    # ajustar cuota bucket 5 para lo que falte (si hard ya ocupó parte)
    # si hard excede target5, dejamos bucket5 = hard y recortamos otros buckets desde el 4 hacia abajo
    if n_hard > targets[5]:
        extra = n_hard - targets[5]
        targets[5] = n_hard
        # recortar extra de 4,3,2,1,0 en ese orden
        for b in [4, 3, 2, 1, 0]:
            if extra <= 0:
                break
            take = min(targets[b], extra)
            targets[b] -= take
            extra -= take
    else:
        targets[5] = targets[5] - n_hard  # lo que queda por llenar con refs libres

    # si por redondeos quedó suma distinta a n_free, ajustar en bucket 4 (neutro)
    total_needed = sum(targets.values())
    if total_needed != n_free:
        diff = n_free - total_needed
        targets[4] = max(0, targets[4] + diff)

    # ordenar por score (menor=mejor)
    free = free.sort_values("score_ref", ascending=True)

    # asignar por bloques
    start = 0
    for b in BUCKETS:
        cnt = targets[b]
        if cnt <= 0:
            continue
        end = start + cnt
        bucket_final.loc[free.index[start:end]] = b
        start = end

# si alguna referencia quedó sin asignar (por negociador nulo o edge cases), cae a bucket por score global
mask_na = bucket_final.isna()
if mask_na.any():
    tmp = ref_table.loc[mask_na].sort_values("score_ref", ascending=True)
    # asignación simple por quantiles globales
    ranks = tmp["score_ref"].rank(method="first")
    q = pd.qcut(ranks, q=6, labels=False, duplicates="drop")
    n_q = int(q.max() + 1) if q.notna().any() else 1
    if n_q > 1:
        q_scaled = np.floor(q * (5 / (n_q - 1))).astype(int)
    else:
        q_scaled = pd.Series(0, index=tmp.index, dtype=int)
    bucket_final.loc[tmp.index] = q_scaled.clip(0, 5).astype("Int64")
    # hard sigue siendo 5
    bucket_final.loc[list(refs_hard_5)] = 5

ref_table["Bucket_ref"] = bucket_final.astype("Int64")

# =========================================================
# 8) PEGAR A TODAS LAS FILAS (UNA REFERENCIA = UN BUCKET)
# =========================================================
df_timeline_final = df_timeline_final.merge(
    ref_table[["Bucket_ref"]],
    left_on="Referencia",
    right_index=True,
    how="left",
)

df_timeline_final["Bucket"] = df_timeline_final["Bucket_ref"].astype("Int64")

# Limpieza auxiliares
df_timeline_final.drop(
    columns=["Bucket_ref", "Bucket_row", "_banco_rank", "ratio_ahorro_db"],
    inplace=True,
    errors="ignore",
)

df_timeline_final["Bucket"] = df_timeline_final["Bucket"].astype(int).clip(0, 5).astype("Int64")

In [38]:
df_timeline_final

,Referencia,Id deuda,Cedula,Nombre del cliente,Negociador,BANCOS_ESTANDAR,Descuento,D_BRAVO,Ingreso_esperado,MORA,...,payment_to_bank,CATEGORIA_PRED,observations,Descuento_Actualizacion,Tipo de Actividad,tipo_fila,Negociador liquidacion,Por?,Pago_banco_esperado,Bucket
0,3012650,111266,5.221621e+07,CLAUDIA PATRICIA AVENDANO CALDERON,Negodito,Scotiabank Colpatria,0.7126,9256000.0,1.177355e+06,120.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.660174e+06,2
1,3122962933,225561,5.086860e+07,Luz Nelly Betancourt Ortega,William Santiago Abril Esguerra,Banco de Occidente,0.5000,2138165.0,1.908312e+05,55.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.069082e+06,2
2,3122962933,225562,5.086860e+07,Luz Nelly Betancourt Ortega,William Santiago Abril Esguerra,Banco de Occidente,0.5600,3299116.0,3.297796e+05,55.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.451611e+06,2
3,3122962933,225563,5.086860e+07,Luz Nelly Betancourt Ortega,William Santiago Abril Esguerra,Tuya,0.6715,5959400.0,7.143101e+05,55.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.957663e+06,2
4,3196892494,404243,5.290289e+07,Diana Marcela Suarez,Dayana Isabel Ojito Ortiz,Banco Popular,0.4800,1921650.0,1.646470e+05,79.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9.992580e+05,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
43173,3225833196,2091518,1.019066e+09,JUAN DAVID LOZANO TRIVINO,Gabriela Saavedra Latorre,Banco Davivienda,0.3637,13695024.0,8.890871e+05,7.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8.714144e+06,4
43174,3225833196,2091519,1.019066e+09,JUAN DAVID LOZANO TRIVINO,Gabriela Saavedra Latorre,Scotiabank Colpatria,0.5000,50463309.0,4.503850e+06,7.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.523165e+07,4
43175,3225833196,2091520,1.019066e+09,JUAN DAVID LOZANO TRIVINO,Gabriela Saavedra Latorre,Bancolombia,0.4700,82022139.0,6.881247e+06,7.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.347173e+07,4
43176,3105507630,2091523,7.938104e+07,JUAN CARLOS PACHECO CONTRERAS,Julian Andres Ossa Bohorquez,Bancolombia,0.4700,130935011.0,1.098479e+07,7.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6.939556e+07,1


In [39]:
import pandas as pd

# 1) Borrar columnas (no falla si alguna no existe)
df_base_funnel = df_base_funnel.drop(columns=["_cat_all_norm", "cat_norm"], errors="ignore")

# 2) Crear mapeo único Id deuda -> Bucket (sin duplicar filas)
#    Si hay repetidos, se queda con el primer Bucket no nulo (y si todos son nulos, queda nulo)
bucket_map = (
    df_timeline_final[["Id deuda", "Bucket"]]
    .dropna(subset=["Id deuda"])
    .sort_values("Id deuda")
    .drop_duplicates(subset=["Id deuda"], keep="first")
)

# 3) Agregar Bucket a df_base_funnel sin crear filas nuevas (LEFT JOIN)
df_base_funnel = df_base_funnel.merge(
    bucket_map,
    on="Id deuda",
    how="left",
    validate="m:1"  # muchas en df_base_funnel -> 1 en bucket_map
)

In [40]:
df_base_funnel

,Referencia,Id deuda,Cedula,Nombre del cliente,Negociador,BANCOS_ESTANDAR,Descuento,D_BRAVO,MORAEstructurable,Potencial,...,es_este_mes,tiene_liquidado_historico,MORA,Estructurable,ultimo contacto,tipo_fila,Negociador liquidacion,Por?,Pago_banco_esperado,Bucket
0,3012650,111266,5.221621e+07,CLAUDIA PATRICIA AVENDANO CALDERON,Negodito,Scotiabank Colpatria,0.7126,9256000.0,<NA>,5.0,...,False,False,120.0,1.0,6.0,NaN,NaN,NaN,2.660174e+06,2
1,3122962933,225561,5.086860e+07,Luz Nelly Betancourt Ortega,William Santiago Abril Esguerra,Banco de Occidente,0.5000,2138165.0,<NA>,6.0,...,False,False,55.0,1.0,1.0,NaN,NaN,NaN,1.069082e+06,2
2,3122962933,225562,5.086860e+07,Luz Nelly Betancourt Ortega,William Santiago Abril Esguerra,Banco de Occidente,0.5600,3299116.0,<NA>,8.0,...,False,False,55.0,1.0,1.0,NaN,NaN,NaN,1.451611e+06,2
3,3122962933,225563,5.086860e+07,Luz Nelly Betancourt Ortega,William Santiago Abril Esguerra,Tuya,0.6715,5959400.0,<NA>,13.0,...,False,False,55.0,0.0,1.0,NaN,NaN,NaN,1.957663e+06,2
4,3196892494,404243,5.290289e+07,Diana Marcela Suarez,Dayana Isabel Ojito Ortiz,Banco Popular,0.4800,1921650.0,<NA>,40.0,...,False,False,79.0,1.0,6.0,NaN,NaN,NaN,9.992580e+05,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41274,3225833196,2091518,1.019066e+09,JUAN DAVID LOZANO TRIVINO,Gabriela Saavedra Latorre,Banco Davivienda,0.3637,13695024.0,<NA>,5.0,...,False,False,7.0,1.0,0.0,NaN,NaN,NaN,8.714144e+06,4
41275,3225833196,2091519,1.019066e+09,JUAN DAVID LOZANO TRIVINO,Gabriela Saavedra Latorre,Scotiabank Colpatria,0.5000,50463309.0,<NA>,15.0,...,False,False,7.0,0.0,0.0,NaN,NaN,NaN,2.523165e+07,4
41276,3225833196,2091520,1.019066e+09,JUAN DAVID LOZANO TRIVINO,Gabriela Saavedra Latorre,Bancolombia,0.4700,82022139.0,<NA>,24.0,...,False,False,7.0,0.0,0.0,NaN,NaN,NaN,4.347173e+07,4
41277,3105507630,2091523,7.938104e+07,JUAN CARLOS PACHECO CONTRERAS,Julian Andres Ossa Bohorquez,Bancolombia,0.4700,130935011.0,<NA>,30.0,...,False,False,7.0,0.0,0.0,NaN,NaN,NaN,6.939556e+07,1


#Subir dataframe

In [41]:
# =========================================================
# SUBIR DATAFRAMES A GOOGLE SHEETS (ULTRA BLINDADO)
# - Timeline y Funnel se suben de forma independiente (no se tumba todo)
# - Reintentos (500/502/503/504/429) + backoff
# - Resize con fallback
# - Fallback automático a chunks (y chunks más pequeños + pausas)
# - “Reopen” del spreadsheet si hay errores persistentes
# =========================================================

import os, json, re, time, random
import numpy as np
import pandas as pd

try:
    import gspread
    from gspread_dataframe import set_with_dataframe
    from google.oauth2.service_account import Credentials
    from gspread.exceptions import APIError, WorksheetNotFound
except ImportError:
    import subprocess, sys
    subprocess.check_call([sys.executable, "-m", "pip", "install", "-q",
                           "gspread", "gspread-dataframe", "google-auth"])
    import gspread
    from gspread_dataframe import set_with_dataframe
    from google.oauth2.service_account import Credentials
    from gspread.exceptions import APIError, WorksheetNotFound


# -----------------------------
# JSON robusto
# -----------------------------
def _robust_json_loads(s: str) -> dict:
    if not isinstance(s, str):
        raise TypeError("Se esperaba string para parsear JSON")
    s0 = s.strip()
    if (s0.startswith('"') and s0.endswith('"')) or (s0.startswith("'") and s0.endswith("'")):
        s0 = s0[1:-1].strip()
    try:
        return json.loads(s0)
    except Exception:
        pass
    s1 = s0.replace("\\n", "\n")
    try:
        return json.loads(s1)
    except Exception:
        pass

    def fix_private_key(text: str) -> str:
        m = re.search(r'"private_key"\s*:\s*"([\s\S]*?)"\s*,\s*"client_email"', text)
        if not m:
            return text
        pk = m.group(1)
        pk_fixed = pk.replace("\n", "\\n")
        return text.replace(pk, pk_fixed)

    s2 = fix_private_key(s0)
    return json.loads(s2)


def load_service_account_info():
    # Colab
    try:
        from google.colab import userdata
        mi_json = userdata.get("MI_JSON")
        if mi_json:
            if isinstance(mi_json, dict):
                return mi_json
            return _robust_json_loads(mi_json)
    except Exception:
        pass

    # GitHub/local: prioriza GOOGLE_SERVICE_ACCOUNT_JSON, si no, MI_JSON
    sa = os.getenv("GOOGLE_SERVICE_ACCOUNT_JSON") or os.getenv("MI_JSON")
    if not sa:
        raise ValueError("❌ Faltan credenciales: define GOOGLE_SERVICE_ACCOUNT_JSON (o MI_JSON) en env/secrets.")
    return _robust_json_loads(sa)


# -----------------------------
# Retry robusto
# -----------------------------
RETRIABLE_CODES = ["[500]", "[502]", "[503]", "[504]", "[429]"]

def _retry(fn, label="", tries=10, base_sleep=1.5, jitter=0.6, max_sleep=45):
    last_err = None
    for i in range(tries):
        try:
            return fn()
        except APIError as e:
            last_err = e
            msg = str(e)
            if any(c in msg for c in RETRIABLE_CODES):
                # resize / update suelen necesitar más aire
                extra = 6.0 if ("resize" in label.lower() or "update" in label.lower()) else 0.0
                sleep_s = min((base_sleep + extra) * (2 ** i) + random.uniform(0, jitter), max_sleep)
                print(f"[RETRY {i+1}/{tries}] {label} -> {msg[:120]}... sleep {sleep_s:.1f}s")
                time.sleep(sleep_s)
                continue
            raise
    raise last_err


# -----------------------------
# Preparar DF
# -----------------------------
def _prepare_df_for_sheets(df: pd.DataFrame) -> pd.DataFrame:
    df_out = df.copy()
    df_out = df_out.replace([np.inf, -np.inf], np.nan)

    for c in df_out.columns:
        if pd.api.types.is_datetime64_any_dtype(df_out[c]):
            df_out[c] = df_out[c].dt.strftime("%Y-%m-%d %H:%M:%S")

    df_out = df_out.where(pd.notna(df_out), "")
    return df_out


# -----------------------------
# Chunks (más estable)
# -----------------------------
def _upload_chunked(ws, df_out: pd.DataFrame, sheet_name: str, chunk_size: int = 3000, pause_s: float = 0.6):
    ncols = max(len(df_out.columns), 1)
    header = [df_out.columns.tolist()]
    _retry(lambda: ws.update(header), label=f"update header {sheet_name}")

    data = df_out.values.tolist()
    start_row = 2
    last_col_letter = gspread.utils.rowcol_to_a1(1, ncols).replace("1", "")

    for i in range(0, len(data), chunk_size):
        chunk = data[i:i + chunk_size]
        end_row = start_row + len(chunk) - 1
        cell_range = f"A{start_row}:{last_col_letter}{end_row}"
        _retry(lambda cr=cell_range, ch=chunk: ws.update(cr, ch, value_input_option="USER_ENTERED"),
               label=f"update chunk {sheet_name} rows {start_row}-{end_row}")
        start_row = end_row + 1
        time.sleep(pause_s)  # bajar presión a la API


# -----------------------------
# Subida principal (con reopen si hace falta)
# -----------------------------
def upload_df_to_sheet_ultra(
    gc,
    spreadsheet_id: str,
    df: pd.DataFrame,
    sheet_name: str,
    prefer_chunked: bool = False,
):
    """
    Devuelve (ok:bool, msg:str)
    """
    try:
        spreadsheet = _retry(lambda: gc.open_by_key(spreadsheet_id), label="open spreadsheet")
        df_out = _prepare_df_for_sheets(df)

        nrows = max(len(df_out) + 1, 2)
        ncols = max(len(df_out.columns), 1)

        def _get_or_create_ws():
            try:
                return spreadsheet.worksheet(sheet_name)
            except WorksheetNotFound:
                return spreadsheet.add_worksheet(title=sheet_name, rows="100", cols=str(ncols))

        ws = _retry(_get_or_create_ws, label=f"get/create worksheet {sheet_name}")
        _retry(lambda: ws.clear(), label=f"clear {sheet_name}")

        # Resize (best effort)
        try:
            _retry(lambda: ws.resize(rows=1000, cols=ncols), label=f"resize small {sheet_name}", tries=10, base_sleep=2.0)
            _retry(lambda: ws.resize(rows=nrows, cols=ncols), label=f"resize final {sheet_name}", tries=10, base_sleep=2.0)
        except APIError as e:
            print(f"⚠️ Resize falló en '{sheet_name}'. Sigo sin resize. {str(e)[:140]}...")

        # Subida
        if not prefer_chunked:
            try:
                _retry(lambda: set_with_dataframe(ws, df_out, include_index=False, include_column_header=True, resize=False),
                       label=f"set_with_dataframe {sheet_name}", tries=8, base_sleep=1.5)
                return True, f"✅ {sheet_name}: {len(df_out):,} filas x {ncols} cols (set_with_dataframe)"
            except APIError as e:
                print(f"⚠️ set_with_dataframe falló en '{sheet_name}'. Paso a chunks. {str(e)[:140]}...")

        # Chunks
        try:
            _upload_chunked(ws, df_out, sheet_name, chunk_size=3000, pause_s=0.6)
            return True, f"✅ {sheet_name}: {len(df_out):,} filas x {ncols} cols (chunked)"
        except APIError as e:
            # Último intento: reabrir spreadsheet y repetir chunked
            print(f"⚠️ Chunked falló en '{sheet_name}'. Re-abro y reintento. {str(e)[:140]}...")
            spreadsheet = _retry(lambda: gc.open_by_key(spreadsheet_id), label="reopen spreadsheet", tries=6, base_sleep=2.0)
            ws = _retry(lambda: spreadsheet.worksheet(sheet_name), label=f"reopen ws {sheet_name}", tries=6, base_sleep=2.0)
            _upload_chunked(ws, df_out, sheet_name, chunk_size=2000, pause_s=0.9)
            return True, f"✅ {sheet_name}: {len(df_out):,} filas x {ncols} cols (chunked retry)"
    except Exception as e:
        return False, f"❌ {sheet_name} falló: {type(e).__name__}: {str(e)[:200]}"


# -----------------------------
# Auth
# -----------------------------
SERVICE_ACCOUNT_INFO = load_service_account_info()
SCOPES = ["https://www.googleapis.com/auth/spreadsheets", "https://www.googleapis.com/auth/drive"]
creds = Credentials.from_service_account_info(SERVICE_ACCOUNT_INFO, scopes=SCOPES)
gc = gspread.authorize(creds)

SPREADSHEET_ID = "1Bm1wjsfXdNDFrFTStQJHkERC08Eo21BwjZnu-WncibY"

# -----------------------------
# Ejecutar sin tumbar todo
# -----------------------------
results = []

ok1, msg1 = upload_df_to_sheet_ultra(gc, SPREADSHEET_ID, df_timeline_final, "Timeline", prefer_chunked=False)
results.append(msg1)

ok2, msg2 = upload_df_to_sheet_ultra(gc, SPREADSHEET_ID, df_base_funnel, "Funnel", prefer_chunked=True)
results.append(msg2)

print("\n".join(results))

# Si quieres que el job NO falle aunque uno falle:
# (en Actions, esto evita exit code 1 si algo no crítico falla)
if not (ok1 and ok2):
    print("⚠️ Hubo fallos en al menos una hoja, pero el proceso terminó.")

/tmp/ipykernel_2204/296600866.py:134: DeprecationWarning: The order of arguments in worksheet.update() has changed. Please pass values first and range_name secondor used named arguments (range_name=, values=)
  _retry(lambda cr=cell_range, ch=chunk: ws.update(cr, ch, value_input_option="USER_ENTERED"),


✅ Timeline: 43,178 filas x 39 cols (set_with_dataframe)
✅ Funnel: 41,279 filas x 47 cols (chunked)
